# Table of Contents

* [Overview](#Overview)
* [Setup](#Setup)
    * [Package installs](#Package-installs)
    * [Authentication and GCP settings](#Authentication-and-GCP-settings)
    * [Imports](#Imports)
    * [Global variables](#Global-variables)
    * [Utils](#Utils)
    * [Updating searchable and retrievable fields](#Updating-searchable-and-retrievable-fields)
* [Data Transformation steps](#Data-Transformation-steps)
    * [Data Analysis](#Data-Analysis)
    * [Simple elements population](#Simple-elements-population)
    * [Brand Extraction](#Brand-Extraction)
    * [Description Generation](#Description-Generation)
    * [Tag Generation](#Tag-Generation)
    * [Color Attribute Extraction](#Color-Attribute-Extraction)
    * [Create and Populate Enhanced Product Table](#Create-and-Populate-Enhanced-Product-Table)
        * [Table cloning and schema extension](#Table-cloning-and-schema-extension)
        * [Data population with extended attributes](#Data-population-with-extended-attributes)
* [Verification of Data enriched catalog](#Verification-of-Data-enriched-catalog)
    * [Data loading](#Data-loading)
    * [Validation](#Validation)
* [Conclusion](#Conclusion)

# Overview

This lab provides a comprehensive guide to managing and enriching product data using Google Cloud services, primarily focusing on Vertex AI and BigQuery. You will learn how to perform essential data operations, from initial setup and authentication to advanced data enrichment using Generative AI.

**Scope:**
*   Fetching and analyzing product data from BigQuery.
*   Utilizing Generative AI (Gemini models) for:
    *   Extracting brand names from product titles.
    *   Generating compelling product descriptions.
    *   Creating relevant product tags.
    *   Consolidating and normalizing product tags.
    *   Identifying color attributes (color families and specific colors).
*   Creating and populating an enhanced product table in BigQuery with the newly generated attributes.
*   Verifying the data enrichment process.
*   Uploading the enriched product data to product catalog.
*   Validating the enriched data through Validation tool of Vertex AI Search for commererce.

**Key Learnings:**
Upon completing this lab, you will be able to:
*   Query and analyze product data stored in BigQuery.
*   Leverage Generative AI models for various product data enrichment tasks, including attribute extraction and content generation.
*   Manage and update BigQuery table schemas and data, including creating tables, copying data, and merging updates.
*   Understand the practical application of AI in enhancing e-commerce product catalogs.

**Steps:**
*   **Setup**: Prepare the Python environment, authenticate with Google Cloud, install required packages, and define global variables and utility functions. This ensures the notebook can interact with GCP services and process data efficiently.
*   **Data Analysis**: Analyze the initial product data from BigQuery to understand its current state, schema, and identify missing attributes (e.g., brand, description, tags, detailed color information) that can be enriched.
*   **Data Enrichment using Generative AI**:
    *   Perform preliminary token analysis on product titles to identify potential values for simpler attributes.
    *   Populate `patterns`, `sizes`, and `audience` fields based on the token analysis.
    *   Use the Gemini model to extract `brand` names from product titles.
    *   Generate compelling `description` fields for products using their titles.
    *   Generate product `tags` based on titles and descriptions, then consolidate these tags to create a more refined and manageable set.
    *   Extract `colorInfo` (color families and specific colors) from product titles.
*   **Store Enriched Data in BigQuery**:
    *   Create a new table (`products_enhanced`) in BigQuery by cloning the original product table.
    *   Extend the schema of this new table to include the newly generated attributes.
    *   Populate the new attributes in the `products_enhanced` table using the data generated in the enrichment phase (from the `df_products` DataFrame).
*   **Verification**:
    *   Load the enriched product data from the `products_enhanced` BigQuery table into a new branch (Branch 2) of the Vertex AI Search for Commerce catalog.
    *   Validate the data enrichment by performing search queries against both the original catalog (Branch 0) and the enriched catalog (Branch 2) using the Vertex AI Search for Commerce console, observing differences in search results and facet availability.



# Setup
Let's get started by preparing your environment. We'll begin with authentication and configuration, which are required for all subsequent API calls.

## Package installs
Install all required Python packages. Run this cell only once after starting a new kernel.

In [11]:
%pip install google google-cloud-retail google-cloud-storage google-cloud-bigquery pandas
%pip install google-cloud-aiplatform google-genai
%pip install google-cloud-bigquery-storage pyarrow tqdm bigquery-magics
%pip install google-cloud-bigquery[pandas] jupyterlab
%pip install fsspec gcsfs
%pip install matplotlib seaborn plotly
%pip install --upgrade ipython-sql

5047.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


5079.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


5100.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


5162.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


5182.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Authentication and GCP settings
Before we can interact with the Retail API, we need to authenticate with Google Cloud and set up our project context. This ensures all API calls are authorized and associated with the correct GCP project. If authentication fails, you'll be prompted to log in interactively. The `project_id` variable will be used throughout the notebook.

**About `project_id` and Application Default Credentials (ADC):**

- **`project_id`**: This uniquely identifies your Google Cloud project. All API requests, resource creation, and billing are tied to this project. Setting the correct `project_id` ensures your operations are performed in the intended environment and resources are properly tracked.

- **Application Default Credentials (ADC)**: ADC is a mechanism that allows your code to automatically find and use your Google Cloud credentials. Running the `gcloud auth application-default login` command sets up ADC by generating credentials that client libraries (like the Retail API) can use to authenticate API calls on your behalf.

**Why this matters:**  
Proper authentication and project selection are essential for secure, authorized access to Google Cloud resources. Without these, API calls will fail or may affect the wrong project. ADC simplifies credential management, especially in development and notebook environments.

In [12]:
import subprocess

try:
  # Try to get an access token
  subprocess.check_output(
    ['gcloud', 'auth', 'application-default', 'print-access-token'],
    stderr=subprocess.STDOUT
  )
  print("Already authenticated with Application Default Credentials.")
except subprocess.CalledProcessError:
  # If it fails, prompt for login
  print("No valid ADC found. Running interactive login...")
  !gcloud auth application-default login

Already authenticated with Application Default Credentials.


## Imports
Import all necessary libraries for API access, data analysis, and visualization.

In [13]:
from google.cloud.retail_v2 import SearchServiceClient, ProductServiceClient, PredictionServiceClient
from google.cloud.retail_v2.types import product, search_service, ListProductsRequest, SearchRequest, PredictRequest, UserEvent
from google.protobuf.field_mask_pb2 import FieldMask
from google.protobuf.json_format import MessageToDict
import pandas as pd
import http.client as http_client
import logging
import re
from IPython.display import display_html
from matplotlib import pyplot as plt
import seaborn as sns

# enabling BigQuery magics
%load_ext bigquery_magics

# configuring default optoins for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Global variables
With authentication complete, let's define some key variables that will be used in all our API calls. These include resource names and placements, which specify the context for search and recommendation requests.

**What is a 'placement'?**  
A placement is a configuration resource in the Retail API that determines how and where a model is used for serving search or recommendation results. Placements define the context (such as search, browse, or recommendation) and can be customized for different pages or user experiences.

**Why might you have multiple placements or branches?**  
- You may have different placements for various parts of your site or app, such as a homepage recommendation carousel, a category browse page, or a personalized search bar.
- Multiple branches allow you to manage different versions of your product catalog (e.g., staging vs. production, or A/B testing different product sets).

**Example scenarios:**
- Using a "default_search" placement for general product search, and a "recently_viewed_default" placement for showing users their recently viewed items.
- Having separate branches for testing new product data before rolling it out to all users, or for running experiments with different recommendation models.

In [14]:
import google.auth
import google.auth.exceptions

# Authenticate with Google Cloud and get the default project ID
try:
  credentials, project_id = google.auth.default()
  print(f"Using project ID: {project_id}")
  !gcloud auth application-default set-quota-project {project_id}
except google.auth.exceptions.DefaultCredentialsError:
  print("Google Cloud Authentication failed. Please configure your credentials.")
  print("You might need to run 'gcloud auth application-default login'")
  project_id = None # Set to None or a default
  
SCRIPTS_BUCKET = "artilekt-vaisc-csb_scripts"

# Define the default placement for search and recommendations
DEFAULT_SEARCH = (
  f"projects/{project_id}/locations/global/catalogs/default_catalog/"
  "placements/default_search" # Use default_search unless you have a specific browse placement
)
RECENTLY_VIEWED_DEFAULT = (
  f"projects/{project_id}/locations/global/catalogs/default_catalog/"
  "placements/recently_viewed_default"
)
DEFAULT_BRANCH = f"projects/{project_id}/locations/global/catalogs/default_catalog/branches/0"

ORIGINAL_PRODUCTS_TABLE = "retail.products"
ENHANCED_PRODUCTS_TABLE = "retail.products_enhanced"

Using project ID: artilekt-vaisc-csb


5218.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



Credentials saved to file: [/home/volenin/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "artilekt-vaisc-csb" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## Utils
To make our analysis easier, we'll use some utility functions for data conversion and HTTP logging. These will help us convert API responses to Pandas DataFrames for analysis and enable detailed logging for troubleshooting.

In [15]:
import pandas as pd
from google.protobuf.json_format import MessageToDict

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def json2df(products_list):
  if products_list:
    products_dicts = [dict(sorted(MessageToDict(p._pb).items())) for p in products_list]
    df = pd.json_normalize(products_dicts)
    return df
  else:
    print("No products returned or an error occurred.")
    return pd.DataFrame()

from contextlib import contextmanager

@contextmanager
def http_logging(log_http: bool):
    """
    Context manager to enable/disable HTTP logging for Google API clients.
    Usage:
        with http_logging(log_http):
            # code that needs HTTP logging
    """
    import http.client as http_client
    import logging
    root_logger = logging.getLogger()
    original_http_debuglevel = http_client.HTTPConnection.debuglevel
    original_log_level = root_logger.level
    try:
        if log_http:
            print("\n--- [INFO] Enabling HTTP Logging (forcing REST transport) ---")
            logging.basicConfig()
            root_logger.setLevel(logging.DEBUG)
            http_client.HTTPConnection.debuglevel = 1
            print("--- [INFO] Using REST transport. ---")
        yield
    finally:
        if log_http:
            http_client.HTTPConnection.debuglevel = original_http_debuglevel
            root_logger.setLevel(original_log_level)
            print("--- [INFO] HTTP Logging & Root Log Level Restored ---")

from google.cloud.bigquery import SchemaField
def flatten_schema_fields(schema, prefix=""):
    """Flattens the BigQuery schema, expanding RECORD types."""
    fields_data = []
    for field in schema:
        field_name_full = f"{prefix}{field.name}"
        if field.field_type == 'RECORD' and field.fields:
            fields_data.append({
                "Field Name": field_name_full,
                "Data Type": field.field_type, # or "STRUCT"
                "Mode": field.mode
            })
            fields_data.extend(flatten_schema_fields(field.fields, prefix=f"{field_name_full}."))
        else:
            fields_data.append({
                "Field Name": field_name_full,
                "Data Type": field.field_type,
                "Mode": field.mode
            })
    return fields_data


# Initialize Vertex AI
from google import genai
import json

def gemini_run_query(contents, model_name="gemini-2.0-flash"):
    """
    Runs a query against a Gemini model and expects a JSON response.

    Args:
        contents: The content to send to the model.
        model_name: The name of the Gemini model to use.
        
    Returns:
        A dictionary parsed from the JSON response, or None if an error occurs.
    """
    # Default config for JSON response
    effective_config = {"response_mime_type": "application/json"}

    gemini = genai.Client(project=project_id, location="us-central1", vertexai=True)

    try:
        response = gemini.models.generate_content( # gemini is the genai.Client instance
            model=model_name, 
            contents=contents,
            config=effective_config, # Changed from generation_config
        )
        # Parse the JSON response
        respText = response.text.strip()
        # print(f"Response text: {respText}") # Uncomment for debugging
        # Convert response to JSON
        return json.loads(respText)
    except Exception as e:
        print(f"Error in gemini_run_query (model: {model_name}, contents starting with: '{str(contents)[:100]}...'): {e}")
        return None # Return None on error

def gemini_run_query_old(contents):
    try:
        response = gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=contents,
            config={
                "response_mime_type": "application/json",
                # "response_schema": list[Recipe],
            },
        )
        # Parse the JSON response
        respText = response.text.strip()
        # print(f"Response text: {respText}")
        # Convert response to JSON
        return json.loads(respText)
    except Exception as e:
        print(f"Error processing request: {e}")
        # Return a dictionary mapping each title in the batch to ["Error"]
        return {}

## Updating searchable and retrievable fields

The Retail API allows you to specify which fields in your product catalog are searchable, indexable, and retrievable. Attribute configuration settings will impact search and recommendations behavior across your site.

Search uses the following attribute settings:

* **Indexable:** Search can filter and facet using this attribute.
* **Dynamic faceting:** Search can automatically use this attribute as a dynamic facet based on past user behavior such as facet clicks and views. To enable dynamic faceting for an attribute, Indexable must be set to true for that attribute.
* **Searchable:** This attribute is searchable by search queries, which increases recall for that attribute. This control is applicable only for text attributes.
* **Retrievable:** If set to true, search returns this attribute in responses to search queries. If all attributes have Retrievable set to false, the search results contain only the product name or (for variants) the product name and color information.

It is recommended that you retrieve only the fields you need for your application to optimize performance and reduce costs. For the purpose of this lab, you will make all fields searchable and retrievable. This is not recommended for production use, but it will simplify demonstration of the effects some of the advanced search configurations have on the results.

Note that changes to indexable, searchable, and retrievable take effect immediately upon your next catalog full ingestion or within 12 hours or more. Changes to dynamic faceting and tiling and exact match take effect within 2 days. Changes to filterable only apply to filter tag generation for recommendations and take effect within 12 hours or more.

You will explore this configuration and effects of these settings in another lab. For the purpose of this lab, we will need to set all attributes to searchable and retrievable. This will allow us to see the effects of the Generative AI enrichment on the search results.

In [42]:
import copy
from google.cloud.retail_v2.types import CatalogAttribute, AttributesConfig, UpdateAttributesConfigRequest
from google.cloud.retail_v2 import CatalogServiceClient

catalog_service_client = CatalogServiceClient()
catalog_name = f"projects/{project_id}/locations/global/catalogs/default_catalog/attributesConfig"
attributes_config = catalog_service_client.get_attributes_config(name=catalog_name)

# Allowed predefined textual attributes for searchable_option
ALLOWED_SEARCHABLE_PREDEFINED = {
    "ageGroups", "brands", "categories", "colorFamilies", "conditions",
    "description", "genders", "materials", "patterns", "sizes", "title"
}

# Define a list of important attributes to be made retrievable (max 30, must be allowed by Retail API)
IMPORTANT_RETRIEVABLE_ATTRIBUTES = [
  "ageGroups",
  "availability",
  "brands",
  "categories",
  "colorFamilies",
  "colors",
  "conditions",
  "cost",
  "currencyCode",
  "description",
  "discount",
  "genders",
  "gtin",
  "images",
  "materials",
  "name",
  "originalPrice",
  "patterns",
  "price",
  "productId",
  "rating",
  "ratingCount",
  "ratingHistogram",
  "sizes",
  "title",
  "uri",
  # Custom attributes below (keep at the bottom)
  "attributes.collection",
  "attributes.ecofriendly",
  "attributes.material",
  "attributes.style",
]

for attr_name, attr in attributes_config.catalog_attributes.items():
    attr.indexable_option = CatalogAttribute.IndexableOption.INDEXABLE_ENABLED
    
    # Set retrievable_option based on importance and inventory status
    if attr_name.startswith("inventories."):
        attr.retrievable_option = CatalogAttribute.RetrievableOption.RETRIEVABLE_DISABLED
    elif attr_name in IMPORTANT_RETRIEVABLE_ATTRIBUTES:
        attr.retrievable_option = CatalogAttribute.RetrievableOption.RETRIEVABLE_ENABLED
    else:
        attr.retrievable_option = CatalogAttribute.RetrievableOption.RETRIEVABLE_DISABLED
        
    # Only set searchable_option=ENABLED for allowed attributes
    if attr_name in ALLOWED_SEARCHABLE_PREDEFINED:
        attr.searchable_option = CatalogAttribute.SearchableOption.SEARCHABLE_ENABLED
    elif attr_name.startswith("attributes.") and attr.type_ == CatalogAttribute.AttributeType.TEXTUAL:
        attr.searchable_option = CatalogAttribute.SearchableOption.SEARCHABLE_ENABLED
    else:
        attr.searchable_option = CatalogAttribute.SearchableOption.SEARCHABLE_DISABLED

# Prepare update request
update_req = UpdateAttributesConfigRequest(
    attributes_config=attributes_config,
    update_mask=None  # None means update all fields
)

# Update the config in the API
updated_config = catalog_service_client.update_attributes_config(request=update_req)

Now that the attributes have been updated, you will purge the catalog Branch 0 to ensure that the changes take effect immediately. This will remove all existing products from the index and trigger a full reindexing with the new attribute settings.

In [ ]:
from google.cloud.retail_v2 import ProductServiceClient
from google.cloud.retail_v2.types import PurgeProductsRequest

# Use the existing product_service_client and project_id variables
branch = DEFAULT_BRANCH

purge_request = PurgeProductsRequest(
  parent=branch,
  filter='*',  # Purge all products
  force=True   # Actually perform the purge (not a dry run)
)

product_service_client = ProductServiceClient()
operation = product_service_client.purge_products(request=purge_request)
print("Purging catalog... This may take a few minutes.")
result = operation.result()
print("Purge operation completed.")
print(result)

> **⚠️ It may take 2-5 minutes for the background process to reload the catalog.** You do not need to wait for this process to complete before proceeding with the rest of the lab.

# Data Transformation steps
We will now proceed with the data transformation steps. 
- First, we will analyze the existing product data loaded into Branch 0 of the product catalog. We will identify any gaps or issues in the data. 
- Then we will see how to address these gaps using data enrichment techniques powered by Generative AI models.
- Finally, we will create a new table in BigQuery with the enriched product data and validate it using the Vertex AI Search for Commerce validation tool.

## Data Analysis
Let's analyze the existing product data loaded into Branch 0 of the product catalog. The data has been imported from a BigQuery table 'products' under 'retail' dataset. Here is the schema and a sample of the data. You can also view the data in the [BigQuery Console](https://console.cloud.google.com/bigquery)

In [16]:
from google.cloud import bigquery
import pandas as pd # Ensure pandas is imported for DataFrame creation

client = bigquery.Client(project=project_id) 

TABLE_ID = f"{project_id}.retail.products"

try:
    table = client.get_table(TABLE_ID)  # Make an API request.
    print(f"Schema for table {TABLE_ID}:")

    # Prepare data for DataFrame using the new flattening function
    schema_data = flatten_schema_fields(table.schema)
    
    df_schema = pd.DataFrame(schema_data)
    display(df_schema)
    
    print(f"\n\nSample data from {TABLE_ID} (first 50 rows):")



    # Query to fetch product titles and IDs
    query = f"""
    SELECT 
        *
    FROM `{project_id}.retail.products`
    WHERE title IS NOT NULL
    LIMIT 50
    """

    print(f"Fetching product data from {project_id}.retail.products...")
    sample_products = client.query(query).to_dataframe()
    print(f"Fetched {len(sample_products)} products")
    display(sample_products)


    # for future use, populate df_products dataframe; we will use this df to add generated information to the products table
    print(f"Fetching distinct product titles from {project_id}.retail.products...")
     # Query to fetch distinct product titles
    query = f"""
    SELECT 
        DISTINCT title
    FROM `{project_id}.retail.products`
    WHERE title IS NOT NULL
    """
    df_products = client.query(query).to_dataframe()

except Exception as e:
    print(f"Error fetching schema or data for table {TABLE_ID}: {e}")

Schema for table artilekt-vaisc-csb.retail.products:


,Field Name,Data Type,Mode
0,name,STRING,NULLABLE
1,id,STRING,REQUIRED
2,primaryProductId,STRING,NULLABLE
3,categories,STRING,REPEATED
4,title,STRING,REQUIRED
5,description,STRING,NULLABLE
6,tags,STRING,REPEATED
7,priceInfo,RECORD,NULLABLE
8,priceInfo.currencyCode,STRING,NULLABLE
9,priceInfo.price,FLOAT,NULLABLE




Sample data from artilekt-vaisc-csb.retail.products (first 50 rows):
Fetching product data from artilekt-vaisc-csb.retail.products...
Fetched 50 products
Fetched 50 products


,name,id,primaryProductId,categories,title,description,tags,priceInfo,availableTime,availability,availableQuantity,uri,images
0,None,GGOEGBJA127699,GGOEGBJA127699,[Bags],Google Mural Tote,None,[],"{'currencyCode': 'USD', 'price': 18.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Mural+Tote,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBJA127699.jpg', 'height': None, 'width': None}]"
1,None,GGOEGBJH124599,GGOEGBJH124599,[Bags],Google Seaport Tote,None,[],"{'currencyCode': 'USD', 'price': 64.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Seaport+Tote,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBJH124599.jpg', 'height': None, 'width': None}]"
2,None,GGOEGBRC128099,GGOEGBRC128099,[Bags],Google Incognito Zippack V2,None,[],"{'currencyCode': 'USD', 'price': 76.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Incognito+Zippack+V2,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBRC128099.jpg', 'height': None, 'width': None}]"
3,None,GGOEGBRC128199,GGOEGBRC128199,[Bags],Google Incognito Laptop Organizer V2,None,[],"{'currencyCode': 'USD', 'price': 56.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Incognito+Laptop+Organizer+V2,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBRC128199.jpg', 'height': None, 'width': None}]"
4,None,GGOEGBJC100199,GGOEGBJC100199,[Bags],Google Mesh Bag Blue,None,[],"{'currencyCode': 'USD', 'price': 6.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Mesh+Bag+Blue,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBJC100199.jpg', 'height': None, 'width': None}]"
5,None,GGCOGBJC100999,GGCOGBJC100999,[Bags],#IamRemarkable Tote,None,[],"{'currencyCode': 'USD', 'price': 8.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/IamRemarkable+Tote,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGCOGBJC100999.jpg', 'height': None, 'width': None}]"
6,None,GGOEGBBC122499,GGOEGBBC122499,[Bags],Google Campus Bike Carry Pouch,None,[],"{'currencyCode': 'USD', 'price': 8.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Google+Campus+Bike+Carry+Pouch,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBBC122499.jpg', 'height': None, 'width': None}]"
7,None,GGOEGBBJ131999,GGOEGBBJ131999,[Bags],Google Cork Pencil Pouch,None,[],"{'currencyCode': 'USD', 'price': 8.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Office/Google+Cork+Pencil+Pouch,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBBJ131999.jpg', 'height': None, 'width': None}]"
8,None,GGOEGBJC122399,GGOEGBJC122399,[Bags],Google Campus Bike Tote Navy,None,[],"{'currencyCode': 'USD', 'price': 11.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Redesign/Bags/Google+Google+Campus+Bike+Tote+Navy,"[{'uri': 'https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBJC122399.jpg', 'height': None, 'width': None}]"
9,None,GGOEGBJD133999,GGOEGBJD133999,[Bags],Google Cambridge Campus Tote,None,[],"{'currencyCode': 'USD', 'price': 11.0, 'originalPrice': None, 'cost': None}",None,IN_STOCK,<NA>,https://shop.googlemerchandisestore.com/Google+Red

Fetching distinct product titles from artilekt-vaisc-csb.retail.products...


Some important fields which impact ranking and searchability of the products are not populated (such as description). You might also notice that some fields are missing entirely from this schema, such as brand and tags. Here is the full schema describing all possible fields that can be populated in the product catalog (You can also view the data in the [BigQuery Console](https://console.cloud.google.com/bigquery), retail.products_tmpl table)

In [17]:
from google.cloud import bigquery
import pandas as pd # Ensure pandas is imported for DataFrame creation

client = bigquery.Client(project=project_id) 

TABLE_ID_TMPL = f"{project_id}.retail.products_tmpl"

try:
    table_tmpl = client.get_table(TABLE_ID_TMPL)  # Make an API request.
    print(f"Schema for table {TABLE_ID_TMPL}:")

    # Prepare data for DataFrame using the new flattening function
    schema_data_tmpl = flatten_schema_fields(table_tmpl.schema)
    
    df_schema_tmpl = pd.DataFrame(schema_data_tmpl)
    display(df_schema_tmpl)

except Exception as e:
    print(f"Error fetching schema or data for table {TABLE_ID_TMPL}: {e}")

Schema for table artilekt-vaisc-csb.retail.products_tmpl:


,Field Name,Data Type,Mode
0,name,STRING,NULLABLE
1,id,STRING,REQUIRED
2,type,STRING,NULLABLE
3,primaryProductId,STRING,NULLABLE
4,collectionMemberIds,STRING,REPEATED
5,gtin,STRING,NULLABLE
6,categories,STRING,REPEATED
7,title,STRING,REQUIRED
8,brands,STRING,REPEATED
9,description,STRING,NULLABLE


Even though `id,` `title` and `category` are the only mandatory fields needed to load catalog data, it is highly desirable to populate as many additional fields as possible to improve the searchability and ranking of the products. Some are more important than others, such as `description`, `brand`, `tags`, `color families`, `target audience`.

If you look at the fields in the original product data, you will notice that the `title` field is the most informative one, which might contain useful information for use to be able to populate additional fields for at least some of the products. There is also a valid reference to `image` uri. In this lab we will focus on working with text fields due to simplicity and low requirement of additional data sources. We will discuss various ways to leverage `image` data at the end of the lab.

Let's analyze tokens in the `title` field and see if we can categorize them against the fields we want to populate. The best approach to analyze unstructured text data is to use Generative AI models, which can help us extract structured information from the text. We will use the Gemini model for this purpose.

In [18]:

from google.cloud import bigquery
import pandas as pd
from collections import Counter
import json
import time
from tqdm import tqdm
import re

# Ensure Gemini client is initialized (should be from previous cells)
# Make sure 'client' is the name of your initialized genai.Client
# client = genai.Client(project=project_id, location="us-central1", vertexai=True)
# The 'gemini_run_query' function uses the 'gemini' client initialized in Cell 12.

# BigQuery client
client_bq = bigquery.Client(project=project_id) 

# 1. Fetch product titles
print("Fetching product titles...")
query_titles = f"SELECT DISTINCT title FROM `{project_id}.retail.products` WHERE title IS NOT NULL"
df_titles = client_bq.query(query_titles).to_dataframe()
print(f"Fetched {len(df_titles)} titles.")

# 2. Fetch target categories from products_tmpl schema
# Assuming df_schema_tmpl is already defined and populated from a previous cell (cell 17)
print("\nUsing pre-existing categories from df_schema_tmpl (from cell 17)...")
target_categories = df_schema_tmpl['Field Name'].tolist()
# Filter out categories that are too generic or complex for simple token mapping if needed
# This list can be refined based on desired output
target_categories = [
    cat for cat in target_categories 
    if not ('.' in cat and not any(sub in cat for sub in ['attributes', 'audience', 'colorInfo', 'priceInfo', 'fulfillmentInfo', 'promotion'])) and 
       cat not in ['name', 'id', 'uri', 'title', 'categories', 'primaryProductId', 'description', 'languageCode', 'gtin', 'retrievableFields', 'expireTime', 'ttl', 'collectionId', 'image']
]
print(f"Using {len(target_categories)} target categories for token mapping: {target_categories}")

# 3. Tokenize titles
print("\nTokenizing titles...")
all_tokens_list = [] # Renamed to avoid conflict if 'all_tokens' is used elsewhere
for title_text in df_titles['title']:
    # Simple tokenization: lowercase and split by non-alphanumeric characters, keeping words
    tokens = re.findall(r'\b\w+\b', str(title_text).lower())
    all_tokens_list.extend(tokens)

unique_tokens_list = list(set(all_tokens_list))
print(f"Found {len(all_tokens_list)} total tokens, {len(unique_tokens_list)} unique tokens.")

# 4. Categorize tokens with Gemini
def categorize_tokens_with_ai(tokens_batch, categories_list):
    prompt = f"""
    You are an expert in e-commerce product data categorization.
    Given a list of product attribute names (categories) and a list of tokens (words from product titles),
    your task is to map each token to the categories from the provided list.

    Categories to use for mapping:
    {json.dumps(categories_list)}

    Rules:
    1. For each token, identify the best matching categories from the list.
    2. A token can belong to multiple categories.
    3. If a token does not clearly fit into any of the provided categories, categorize it as "Uncategorized".
    4. Return the result as a JSON map where the key is the token and the value is the assigned category string.
    5. Be precise. For example, if categories include "attributes.brand" and "attributes.color", a token like "Nike" should map to "attributes.brand", and "red" to "attributes.color".
    6. If a token seems like a general descriptor not fitting a specific attribute (e.g., "new", "sale", "for"), map it to "Uncategorized".

    Tokens to categorize:
    {json.dumps(tokens_batch)}
    """
    # Call the generic gemini_run_query function
    categorized_map = gemini_run_query(
            contents=prompt,

    )

    if categorized_map is None: # Error occurred in gemini_run_query
            # gemini_run_query already prints a detailed error.
            # This function needs to return the specific error structure it promised.
            print(f"Error categorizing batch (tokens starting with '{tokens_batch[0] if tokens_batch else ''}'). AI query failed.")
            return {token: "Error" for token in tokens_batch}

    return categorized_map

print("\nCategorizing tokens with Gemini...")
batch_size_categorization = 150 # Adjusted batch size to manage prompt length and complexity
master_token_category_map = {}

if unique_tokens_list:
    for i in tqdm(range(0, len(unique_tokens_list), batch_size_categorization)):
        current_tokens_batch = unique_tokens_list[i:i+batch_size_categorization]
        batch_map = categorize_tokens_with_ai(current_tokens_batch, target_categories)
        master_token_category_map.update(batch_map) # batch_map will be {token:"Error",...} or {} on error, or valid map
        time.sleep(1.5) # Increased sleep time slightly for API rate limits with potentially larger prompts
else:
    print("No unique tokens to categorize.")

print(f"Finished categorizing tokens. {len(master_token_category_map)} unique tokens processed.")

# 5. Aggregate results
print("\nAggregating results...")
# Initialize with all target categories plus Uncategorized and Error
category_token_counts = {cat: Counter() for cat in target_categories + ["Uncategorized", "Error"]}

for token_item in all_tokens_list: # Iterate through all tokens (with duplicates) to get correct counts
    # Get the category for the token, defaulting to "Uncategorized" if not in map or if mapping failed
    category_for_token = master_token_category_map.get(token_item, "Uncategorized")
    
    # Ensure the category is one we are tracking, otherwise default to Uncategorized
    if category_for_token not in category_token_counts:
        # This case handles if AI returns a category not in target_categories or if it's "Error" from categorize_tokens_with_ai
        if category_for_token == "Error": # Explicitly count tokens that resulted in an "Error" category
            category_token_counts["Error"][token_item] +=1
            continue # Skip further processing for this error token
        else: # For any other unexpected category, map to "Uncategorized"
            category_for_token = "Uncategorized"
        
    category_token_counts[category_for_token][token_item] += 1


Fetching product titles...
Fetched 451 titles.

Using pre-existing categories from df_schema_tmpl (from cell 17)...
Using 37 target categories for token mapping: ['type', 'collectionMemberIds', 'brands', 'attributes', 'attributes.key', 'attributes.value', 'attributes.value.text', 'attributes.value.numbers', 'tags', 'priceInfo', 'priceInfo.currencyCode', 'priceInfo.price', 'priceInfo.originalPrice', 'priceInfo.cost', 'priceInfo.priceEffectiveTime', 'priceInfo.priceExpireTime', 'rating', 'availableTime', 'availability', 'availableQuantity', 'fulfillmentInfo', 'fulfillmentInfo.type', 'fulfillmentInfo.placeIds', 'images', 'audience', 'audience.genders', 'audience.ageGroups', 'colorInfo', 'colorInfo.colorFamilies', 'colorInfo.colors', 'sizes', 'materials', 'patterns', 'conditions', 'publishTime', 'promotions', 'promotions.promotionId']

Tokenizing titles...
Found 1966 total tokens, 376 unique tokens.

Categorizing tokens with Gemini...
Fetched 451 titles.

Using pre-existing categories from

100%|██████████| 3/3 [00:23<00:00,  7.84s/it]

Finished categorizing tokens. 376 unique tokens processed.

Aggregating results...


In [19]:

# 6. Format output
print("\nSummary of token categorization:")
summary_data_list = [] # Renamed to avoid conflict
for category_name, token_counter_obj in category_token_counts.items():
    if not token_counter_obj: # Skip categories with no tokens assigned
        continue
    total_tokens_in_category = sum(token_counter_obj.values())
    # Get (token, count) for top 50, then extract just the token string
    tokens_list = [item[0] for item in token_counter_obj.most_common(50)]
    summary_data_list.append({
        "category": category_name,
        "token_count": total_tokens_in_category,
        "tokens_list": tokens_list
    })

# Sort by token count in descending order
df_summary_output = pd.DataFrame(summary_data_list)
if not df_summary_output.empty:
    df_summary_output = df_summary_output.sort_values(by="token_count", ascending=False).reset_index(drop=True)
    print("Category, Token Count, [Top 50 Tokens]")
    display(df_summary_output)
else:
    print("No data to summarize.")

print("\nToken analysis and categorization complete.")



Summary of token categorization:
Category, Token Count, [Top 50 Tokens]
Category, Token Count, [Top 50 Tokens]


,category,token_count,tokens_list
0,type,529,"[tee, bottle, hoodie, zip, tote, pin, sticker, mug, set, journal, patch, bag, crew, bike, pad, hat, sweatshirt, mural, pouch, cap, tumbler, sheet, confetti, notebook, onesie, backpack, pack, beanie, sock, sunglasses, socks, blanket, jacket, camp, cup, frisbee, collar, leash, laptop, jotter, phone, longsleeve, pullover, crewneck, vest, water, sculpture, holder, strap, organizer]"
1,Uncategorized,529,"[campus, android, pen, iconic, youtube, lapel, cloud, sea, iamremarkable, chicago, austin, boulder, la, pnw, eco, land, pet, mountain, view, sf, incognito, task, hero, and, nyc, v2, dino, fc, friends, seattle, sunnyvale, french, tech, super, badge, beekeepers, stan, supernatural, earth, day, up, icon, cleaner, utility, trace, natural, picnic, run, 2020, vintage]"
2,brands,391,"[google, cambridge, kirkland, tudes, stojo, nest, moleskine, cotopaxi, keepcup, starter, carhartt, nalgene]"
3,colorInfo.colors,198,"[black, navy, green, blue, white, grey, red, charcoal, yellow, gray, olive, clear, shades, pink, moss, oatmeal, citron, neon, coral, lilac, maroon, mint, ash, heather, ivory, multicolored, beige, metallic]"
4,materials,78,"[cork, chrome, terry, fleece, paper, recycled, felt, twill, sherpa, glass, cardboard, mesh, hemp, appeel, bamboo, cotton, microfleece, shell, softshell, stainless, straw, wooden, leather, heathered, combed, modal, aluminum, tritan, corkbase, stoneware, cloth, blend, woodtop]"
5,sizes,65,"[s, large, f, c, small, g, medium, standard, t, v, sm, mini, jumbo, w, lt, 20oz]"
6,audience.genders,62,"[unisex, women, ladies, men, womens, mommy, daddy, mens]"
7,attributes,34,"[pocket, soft, light, thermal, 4in, flex, slim, refillable, premium, tonal, straight, dark, packable, sleeve, neck, heavyweight]"
8,audience.ageGroups,31,"[youth, toddler, kids, infant, baby]"
9,patterns,17,"[striped, speckled, grid, lines, print, rainbow, snowflake, dot]"



Token analysis and categorization complete.


Out of the categorized tokens, some can be ignored (such as 'type, uncategorized), while others can be used to populate the fields we want to enrich. For example, tokens 'Nike', 'Adidas', 'Puma' can be used to populate the `brand` field, while tokens like 'running', 'sports', 'casual' can be used to populate the `tags` field. Field types that seem to contain useful information are
- `colorInfo.colors`
- `brands`
- `audience`
- `materials`
- `patterns`
- `sizes`

Now that we understand the existing product data and the fields we want to enrich, let's proceed with the data enrichment steps. We will use Generative AI models to extract structured information from the `title` field and populate some of the fields in the product catalog:
- `colorInfo`
- `brands`
- `description`
- `tags`

But first, we will populate some of the fields were values seem to have been identified correctly by preliminary token based analysis. These fields are:
- `patterns`
- `sizes`
- `audience`

## Simple elements population

By initial analysis of the product titles, we have already extraced some useful information that can be used to populate the `patterns`, `sizes`, and `audience` fields. Let's populate them using simple string matching techniques.

In [20]:
# print tokens for patterns, sizes, audiences
display(df_summary_output[df_summary_output['category'].str.contains('pattern|size|audience.', regex=True, case=False)])

# populate patters, sizes and audiences in df_products if any word in the okens_list is found in the title

# Helper function to get all tokens for a given category keyword
def get_tokens_for_category(df_summary, category_keyword):
    relevant_rows = df_summary[df_summary['category'].str.contains(category_keyword, regex=True, case=False)]
    all_tokens = set()
    for tokens_list in relevant_rows['tokens_list']:
        for token in tokens_list:
            all_tokens.add(str(token).lower())
    return list(all_tokens)

# Get dynamic lists of tokens
pattern_tokens = get_tokens_for_category(df_summary_output, 'pattern')
size_tokens = get_tokens_for_category(df_summary_output, 'size')
# For audience, we might want to be more specific if 'audience.' matches too broadly
# or combine 'audience.genders' and 'audience.ageGroups'
audience_gender_tokens = get_tokens_for_category(df_summary_output, 'audience.genders')
audience_age_tokens = get_tokens_for_category(df_summary_output, 'audience.ageGroups')
audience_tokens = {
    "genders": audience_gender_tokens,
    "ageGroups": audience_age_tokens
}

print(f"Pattern tokens: {pattern_tokens}")
print(f"Size tokens: {size_tokens}")
print(f"Audience tokens: {audience_tokens}")

def extract_patterns(title):
    patterns = []
    # Ensure pattern_tokens is not empty before proceeding
    if not pattern_tokens:
        return None
    for token in title.split():
        if token.lower() in pattern_tokens:
            patterns.append(token)
    return ', '.join(patterns) if patterns else None

def extract_sizes(title):
    sizes = []
    # Ensure size_tokens is not empty
    if not size_tokens:
        return None
    for token in title.split():
        if token.lower() in size_tokens:
            sizes.append(token)
    return ', '.join(sizes) if sizes else None

def extract_audiences(title):
    """Extract audience information including genders and ageGroups from title"""
    result = {
        "genders": [],
        "ageGroups": []
    }
    
    # Ensure audience_tokens is not empty and has expected structure
    if not audience_tokens or not isinstance(audience_tokens, dict):
        return None
    
    title_tokens = [token.lower() for token in title.split()]
    
    # Check for gender tokens
    if "genders" in audience_tokens and audience_tokens["genders"]:
        for token in title_tokens:
            if token in audience_tokens["genders"]:
                result["genders"].append(token)
    
    # Check for age group tokens
    if "ageGroups" in audience_tokens and audience_tokens["ageGroups"]:
        for token in title_tokens:
            if token in audience_tokens["ageGroups"]:
                result["ageGroups"].append(token)
    
    # Return None if no audience data found, otherwise return the structured result
    if not result["genders"] and not result["ageGroups"]:
        return None
    
    return result


df_products['pattern'] = df_products['title'].apply(lambda x: extract_patterns(x))
df_products['size'] = df_products['title'].apply(lambda x: extract_sizes(x))
df_products['audience'] = df_products['title'].apply(lambda x: extract_audiences(x))

,category,token_count,tokens_list
5,sizes,65,"[s, large, f, c, small, g, medium, standard, t, v, sm, mini, jumbo, w, lt, 20oz]"
6,audience.genders,62,"[unisex, women, ladies, men, womens, mommy, daddy, mens]"
8,audience.ageGroups,31,"[youth, toddler, kids, infant, baby]"
9,patterns,17,"[striped, speckled, grid, lines, print, rainbow, snowflake, dot]"


Pattern tokens: ['snowflake', 'print', 'grid', 'dot', 'striped', 'speckled', 'rainbow', 'lines']
Size tokens: ['t', 'lt', 's', 'v', 'standard', 'mini', 'g', 'sm', 'w', '20oz', 'c', 'small', 'jumbo', 'medium', 'large', 'f']
Audience tokens: {'genders': ['men', 'unisex', 'ladies', 'mommy', 'daddy', 'mens', 'women', 'womens'], 'ageGroups': ['kids', 'youth', 'toddler', 'baby', 'infant']}


In [21]:

display(df_products[
    df_products['pattern'].notna() | 
    df_products['size'].notna() | 
    df_products['audience'].notna()
][['title', 'pattern', 'size', 'audience']].head(10))

,title,pattern,size,audience
17,Google Large Tote White,None,Large,None
25,Google Bike Mini Backpack,None,Mini,None
48,Android Small Trace Journal Black,None,Small,None
49,Google Campus Bike Grid Task Pad,Grid,None,None
51,Android Large Trace Journal Black,None,Large,None
52,Google Large Standard Journal Grey,None,"Large, Standard",None
71,Google Red Kids Sunglasses,None,None,"{'genders': [], 'ageGroups': ['kids']}"
72,Google Blue Kids Sunglasses,None,None,"{'genders': [], 'ageGroups': ['kids']}"
78,#IamRemarkable Ladies T-Shirt,None,None,"{'genders': ['ladies'], 'ageGroups': []}"
86,Google Crew Striped Athletic Sock,Striped,None,None


## Brand Extraction

Now let's use Vertex AI's Generative AI capabilities to extract brand information from product titles. We'll use the Gemini model to analyze titles and identify potential brands.

In [22]:
from google import genai
from pydantic import BaseModel
import json
import time


from tqdm import tqdm



def extract_brands_with_ai(titles_batch, suggested_brands):
    """
    Extract brands from a batch of product titles using Generative AI,
    leveraging a list of suggested brands from prior tokenization.
    """
    
    # Create prompt for brand extraction
    prompt = f"""
    You are an expert in e-commerce product data analysis. Your task is to extract brand names from product titles.

    Consider these potential brand names, which were identified during an initial analysis of product titles. Use them as strong suggestions:
    {json.dumps(suggested_brands)}

    Rules:
    1. Extract only actual brand names, not product types or categories.
    2. Use the provided list of potential brand names as strong suggestions. However, also identify other brands if they are clearly present in the title but not on the suggestion list.
    3. Return results as a JSON map (NOT array of maps) where the key is the original title and the value is an array of identified brand names (e.g., {{"title": ["BrandA", "BrandB"]}}).
    4. If a title contains a single brand, return that brand as a single element within an array (e.g., {{"title": ["BrandA"]}}).
    5. If no brand is identifiable in a title, return "Unknown" as a single element in an array for that title (e.g., {{"title": ["Unknown"]}}).
    6. If a title contains multiple brands, return all identified brands in an array, in the order they appear in the title (e.g., {{"title": ["BrandX", "BrandY"]}}).
    7. Be consistent with brand name formatting. If a brand matches one from the suggested list, prefer the casing from that list. Otherwise, use standard capitalization (e.g., "Google", not "google").
    8. Ignore generic terms, product types, or attributes that are not brand names.
    
    Product titles to analyze:
    {chr(10).join([f"{i+1}. {title}" for i, title in enumerate(titles_batch)])}
    """
    
    try:
        # print(prompt)
        # Call the gemini_run_query function
        brands_map = gemini_run_query(
            contents=prompt,
        )
        print(f"brands: {brands_map}")
        if brands_map is None: # Error occurred in gemini_run_query
            # gemini_run_query already prints a detailed error.
            # This function needs to return the specific error structure it promised.
            print(f"AI query failed for titles starting with '{titles_batch[0] if titles_batch else 'N/A'}'.")
            return {title: ["Error"] for title in titles_batch}
            
        return brands_map
    except Exception as e: # Catch any other unexpected errors during the process
        print(f"Unexpected error in extract_brands_with_ai for titles starting with '{titles_batch[0] if titles_batch else 'N/A'}': {e}")
        return {title: ["Error"] for title in titles_batch}

# Process titles in batches
print("=== Extracting Brands using Generative AI ===")

# Prepare suggested brands from earlier tokenization (cell 18)
suggested_brands_from_tokenization = []
if 'category_token_counts' in globals() and isinstance(category_token_counts, dict) and 'brands' in category_token_counts:
    suggested_brands_from_tokenization = list(category_token_counts['brands'].keys())
    print(f"Using {len(suggested_brands_from_tokenization)} suggested brands from tokenization analysis.")
    # Optional: Display a sample of suggested brands
    # print(f"Sample of suggested brands: {suggested_brands_from_tokenization[:10]}")
else:
    print("Warning: 'category_token_counts' not found or 'brands' key missing. Proceeding without brand suggestions for the AI.")


batch_size = 200  # Small batch size to avoid API limits
final_merged_brands_map = {} # To store all brand maps from all batches

# Get unique titles for processing
unique_titles = df_products['title'].unique().tolist()
print(f"Processing {len(unique_titles)} unique product titles in batches of {batch_size}...")

for i in tqdm(range(0, len(unique_titles), batch_size)):
    print(f"Processing batch {i // batch_size + 1}...")
    current_batch_titles = unique_titles[i:i+batch_size]
    
    # extract_brands_with_ai is expected to return a dict: {title: [brands], ...}
    brands_data_for_batch = extract_brands_with_ai(current_batch_titles, suggested_brands_from_tokenization)
    
    # Update the main dictionary with data from the current batch
    if isinstance(brands_data_for_batch, dict):
        final_merged_brands_map.update(brands_data_for_batch)
    else:
        # Fallback if extract_brands_with_ai doesn't return a dict
        print(f"Warning: AI processing for batch starting with '{current_batch_titles[0] if current_batch_titles else 'N/A'}' did not return a dictionary. These titles may be missing brand data or have errors.")
        for title_in_batch in current_batch_titles:
            final_merged_brands_map.setdefault(title_in_batch, ["Error - Batch Data Invalid"])
    
    time.sleep(1)


# Assign the extracted brands to the DataFrame
# Map titles to their extracted brand lists; titles not in map get NaN
df_products['brand'] = df_products['title'].map(final_merged_brands_map)

# Ensure that the 'brand' column contains lists.
# If map resulted in NaN (title not found) or if value is not a list, set to ["Unknown"].
df_products['brand'] = df_products['brand'].apply(
    lambda x: x if isinstance(x, list) else ["Unknown"]
)
print("Brand extraction completed!")


=== Extracting Brands using Generative AI ===
Using 12 suggested brands from tokenization analysis.
Processing 451 unique product titles in batches of 200...


  0%|          | 0/3 [00:00<?, ?it/s]

Processing batch 1...
brands: {'Google Mural Tote': ['Google'], 'Google Seaport Tote': ['Google'], 'Google Incognito Zippack V2': ['Google'], 'Google Incognito Laptop Organizer V2': ['Google'], 'Google Mesh Bag Blue': ['Google'], '#IamRemarkable Tote': ['Unknown'], 'Google Campus Bike Carry Pouch': ['Google'], 'Google Cork Pencil Pouch': ['Google'], 'Google Campus Bike Tote Navy': ['Google'], 'Google Cambridge Campus Tote': ['Google', 'cambridge'], 'Google Chicago Campus Tote': ['Google'], 'Google Austin Campus Tote': ['Google'], 'Google Boulder Campus Tote': ['Google'], 'Google LA Campus Tote': ['Google'], 'Google PNW Campus Tote': ['Google'], 'Google Confetti Accessory Pouch': ['Google'], 'Google Confetti Tote White': ['Google'], 'Google Large Tote White': ['Google'], 'Google Hemp Tote': ['Google'], 'Google Totepak': ['Google'], 'Supernatural Paper Lunch Sac': ['Unknown'], 'Android Iconic Backpack': ['Android'], 'Supernatural Paper Tote': ['Unknown'], 'Google Flat Front Bag Grey': ['

 33%|███▎      | 1/3 [00:12<00:25, 12.72s/it]

Processing batch 2...
brands: {'Google LA Campus Zip Hoodie': ['Google'], 'Google SF Campus Zip Hoodie': ['Google'], 'Google PNW Campus Zip Hoodie': ['Google'], 'Google Sunnyvale Campus Zip Hoodie': ['Google'], 'Google Zip Hoodie F/C': ['Google'], 'Google Campus Unisex Zip Hoodie': ['Google'], 'YouTube Standards Zip Hoodie Black': ['YouTube'], 'Google Summer19 Crew Grey': ['Google'], 'Google Campus Raincoat Green': ['Google'], 'Google Raincoat Navy': ['Google'], 'Google Black Cloud Zip Hoodie': ['Google'], 'Google Sherpa Zip Hoodie Navy': ['Google'], 'Google Sherpa Zip Hoodie Charcoal': ['Google'], "Google Women's Tech Fleece Grey": ['Google'], 'Google Toddler FC Zip Hoodie': ['Google'], 'Super G Unisex Joggers': ['Unknown'], 'Google Youth FC Zip Hoodie': ['Google'], 'Google Marine Layer Tee': ['Google'], 'Google Soft Modal Scarf': ['Google'], 'Google Land & Sea French Terry Sweatshirt': ['Google'], 'Google Crewneck Sweatshirt Navy': ['Google'], 'Google Crewneck Sweatshirt Grey': ['Goo

 67%|██████▋   | 2/3 [00:25<00:12, 12.49s/it]

Processing batch 3...
brands: {'YouTube Leather Strap Hat Black': ['YouTube'], 'Google Land & Sea Cotton Cap': ['google'], 'Google Leather Strap Hat Black': ['google'], 'Google Trike Tee Black': ['google'], 'YouTube Kids Tee Black': ['YouTube'], 'Google Speckled Beanie Grey': ['google'], 'Android Pocket Onesie Navy': ['Android'], 'Womens Google Black Tee': ['google'], 'Unisex Google Grey Tee': ['google'], "Google Women's Tee F/C Black": ['google'], 'Google Blue Bandana Bib': ['google'], 'Google Unisex V-neck Tee': ['google'], 'Google Striped Tank': ['google'], 'Google Cloud Carhartt Crew Sweatshirt': ['google', 'Carhartt'], 'Google Cloud Tri-Blend Crew Tee': ['google'], 'Google Vintage Cap Navy': ['google'], 'Google Tracking Hat': ['google'], 'Super G French Terry Sweatshirt': ['Unknown'], 'Google Badge Heavyweight Pullover Black': ['google'], 'Google Mountain View Campus Zip Hoodie': ['google'], 'Google Black Eco Zip Hoodie': ['google'], "Google Women's Discovery Lt. Rain Shell": ['go

100%|██████████| 3/3 [00:29<00:00,  9.88s/it]

Brand extraction completed!


In [23]:
display(df_products[['title', 'brand']].head(10))

,title,brand
0,Google Mural Tote,[Google]
1,Google Seaport Tote,[Google]
2,Google Incognito Zippack V2,[Google]
3,Google Incognito Laptop Organizer V2,[Google]
4,Google Mesh Bag Blue,[Google]
5,#IamRemarkable Tote,[Unknown]
6,Google Campus Bike Carry Pouch,[Google]
7,Google Cork Pencil Pouch,[Google]
8,Google Campus Bike Tote Navy,[Google]
9,Google Cambridge Campus Tote,"[Google, cambridge]"


## Description Generation
Next, we'll generate product descriptions using the Gemini model. This will help us create compelling and informative descriptions for each product based on its title and other attributes.

In [24]:
from google import genai
import json
import time
from tqdm import tqdm

# Ensure client is initialized (it should be from the previous cell, but good for clarity)
# client = genai.Client(project=project_id, location="us-central1", vertexai=True)

def generate_descriptions_with_ai(titles_batch):
    """
    Generate product descriptions from a batch of product titles using Generative AI.
    """
    prompt = f"""
    You are an expert e-commerce copywriter. Your task is to generate compelling and concise product descriptions 
    suitable for a retail portal based on product titles.

    Rules:
    1. For each product title, generate a description that is 2-4 sentences long.
    2. Highlight key features and benefits implied by the title.
    3. Use engaging and persuasive language.
    4. Return results as a JSON map (NOT as an array of maps) where the key is the original product title and the value is the generated product description string.
    5. If a product title is too generic or uninformative to generate a meaningful description, use title as description.
    6. Ensure the output is a valid JSON object.

    Product titles to analyze:
    {chr(10).join([f"{i+1}. {title}" for i, title in enumerate(titles_batch)])}
    """

    try:
        # Call the gemini_run_query function, which handles model selection,
        # JSON response type, and basic error handling.
        descriptions_map = gemini_run_query(contents=prompt)

        if descriptions_map is None: # Error occurred in gemini_run_query
            # gemini_run_query already prints a detailed error.
            # This function needs to return the specific error structure it promised.
            print(f"AI query failed for titles starting with '{titles_batch[0] if titles_batch else 'N/A'}'.")
            return {title: "Error generating description" for title in titles_batch}
            
        return descriptions_map
    except Exception as e: # Catch any other unexpected errors during the process
        print(f"Unexpected error in generate_descriptions_with_ai for titles starting with '{titles_batch[0] if titles_batch else 'N/A'}': {e}")
        return {title: "Error generating description" for title in titles_batch}

# Process titles for descriptions in batches
print("\n=== Generating Product Descriptions using Generative AI ===")
batch_size = 100  # Adjust batch size as needed, considering prompt length and API limits for descriptions
final_merged_descriptions_map = {}

# Get unique titles for processing
unique_titles_for_description = df_products['title'].unique().tolist()
print(f"Processing {len(unique_titles_for_description)} unique product titles for descriptions in batches of {batch_size}...")

for i in tqdm(range(0, len(unique_titles_for_description), batch_size)):
    print(f"Processing description batch {i // batch_size + 1}...")
    current_batch_titles = unique_titles_for_description[i:i+batch_size]
    
    descriptions_data_for_batch = generate_descriptions_with_ai(current_batch_titles)
    
    if isinstance(descriptions_data_for_batch, dict):
        final_merged_descriptions_map.update(descriptions_data_for_batch)
    else:
        print(f"Warning: AI processing for description batch starting with '{current_batch_titles[0] if current_batch_titles else 'N/A'}' did not return a dictionary.")
        for title_in_batch in current_batch_titles:
            final_merged_descriptions_map.setdefault(title_in_batch, "Error - Batch Data Invalid")
    
    time.sleep(1) # Respect API rate limits

# Assign the generated descriptions to the DataFrame
df_products['description'] = df_products['title'].map(final_merged_descriptions_map)

# Ensure that the 'description' column contains strings, default if not.
df_products['description'] = df_products['description'].apply(
    lambda x: x if isinstance(x, str) else "Description not available"
)



=== Generating Product Descriptions using Generative AI ===
Processing 451 unique product titles for descriptions in batches of 100...


  0%|          | 0/5 [00:00<?, ?it/s]

Processing description batch 1...


 20%|██        | 1/5 [00:30<02:00, 30.19s/it]

Processing description batch 2...


...

 60%|██████    | 3/5 [01:14<00:47, 23.96s/it]

Processing description batch 4...


 80%|████████  | 4/5 [01:36<00:23, 23.00s/it]

Processing description batch 5...


100%|██████████| 5/5 [01:50<00:00, 22.06s/it]


In [25]:
print("\nProduct description generation completed!")
display(df_products[['title', 'description']].head(10))


Product description generation completed!


,title,description
0,Google Mural Tote,"Carry your essentials in style with the Google Mural Tote. Featuring a unique mural design, this tote is both fashionable and functional. Perfect for everyday use, it's a great way to show off your Google pride."
1,Google Seaport Tote,"Show your Google spirit with the Google Seaport Tote. This stylish and durable tote bag is perfect for carrying your everyday essentials. Its spacious design and comfortable straps make it ideal for work, school, or travel."
2,Google Incognito Zippack V2,Keep your belongings secure and organized with the Google Incognito Zippack V2. This updated design offers enhanced security with its zippered compartments. Ideal for those who value privacy and functionality.
3,Google Incognito Laptop Organizer V2,The Google Incognito Laptop Organizer V2 offers discreet protection and organization for your laptop and accessories. This updated version provides enhanced features and a sleek design. Perfect for professionals and students alike who need to keep their tech safe.
4,Google Mesh Bag Blue,"The Google Mesh Bag in Blue is a lightweight and breathable option for carrying your gear. Perfect for the gym, beach, or everyday errands. Its mesh design allows for ventilation and easy visibility of contents."
5,#IamRemarkable Tote,Celebrate the #IamRemarkable initiative with this empowering tote bag. Perfect for carrying your daily essentials while promoting self-promotion and confidence. Show your support and spread the message wherever you go.
6,Google Campus Bike Carry Pouch,The Google Campus Bike Carry Pouch is designed for convenient storage while cycling. Securely carry your essentials on your bike with this durable and easy-to-install pouch. Perfect for campus commuters and urban cyclists.
7,Google Cork Pencil Pouch,"Store your writing essentials in sustainable style with the Google Cork Pencil Pouch. Made from eco-friendly cork, this pouch is both durable and unique. Keep your pens and pencils organized in a responsible way."
8,Google Campus Bike Tote Navy,"The Google Campus Bike Tote in Navy is designed for easy and stylish transportation on your bike. This tote securely attaches to your bike rack, making it perfect for carrying groceries, books, or other essentials. Enjoy a convenient and eco-friendly way to commute."
9,Google Cambridge Campus Tote,"Show your Cambridge campus pride with this durable and stylish tote. Perfect for carrying books, laptops, and other essentials around campus. Made with high-quality materials for long-lasting use."


## Tag Generation
Now, let's generate product tags using the Gemini model. Tags are essential for improving searchability and categorization of products in the catalog. We'll use the model to create relevant tags based on product titles and descriptions. We will also incorporate the brand information extracted earlier to ensure tags are relevant and comprehensive.

In [26]:
from google import genai
import json
import time
from tqdm import tqdm
import re # Ensure re is imported

# Ensure client is initialized
# client = genai.Client(project=project_id, location="us-central1", vertexai=True)

def generate_tags_with_ai(batch_data, suggested_tags):
    """
    Generate tags from a batch of product titles, descriptions, and brands using Generative AI.
    Brands (provided in batch_data) are removed from the generated tags during post-processing.
    Suggested tags are provided to the AI as a hint.
    Batch data should be a list of dictionaries, each with 'title', 'description', and 'brand'.
    """

    tag_rules = """
      Tag best practices:
      1. Tag Length: The ideal length for a tag is typically 1 to 3 words.

      The goal is to capture the essence of a searchable keyword or a specific attribute. Think about what a user would realistically type into a search bar or what would make sense as a clickable filter in a sidebar.

      Good Examples (1-3 words):
      - organic cotton
      - water-resistant
      - summer collection
      - vintage wash
      - limited edition
      - graphic print


      2. What to Avoid:
      - Too Broad (often 1 word): A tag like "shirt" is often too generic if the product is already in a "Shirts" category. It adds very little new information.
      - Too Long (like a sentence): A tag like "this t-shirt is made from premium sustainable cotton" is not a tag; it's a description. It's ineffective for faceting and won't match user search behavior.
    """
    prompt_parts = [
        "You are an expert e-commerce product tagger. Your task is to generate a list of 5 to 10 relevant tags for each product based on its title and description.",
        "Focus on keywords that customers might use to search for these products.",
        "Tags should be concise and can include product attributes, category, use cases, or key features.",
        tag_rules,
        "Consider these potential tags, which were identified during an initial analysis. Use them as strong suggestions if applicable, but also generate other relevant tags:",
        json.dumps(suggested_tags),
        "Return results as a JSON map where the key is the original product title and the value is an array of generated tag strings.",
        "If a product title or description is too generic or uninformative to generate meaningful tags, return an empty array for that title.",
        "Ensure the output is a valid JSON object.",
        "\nProduct data to analyze:"
    ]
    
    for i, item in enumerate(batch_data):
        prompt_parts.append(f"{i+1}. Title: {item['title']}\n   Description: {item['description']}")
        
    prompt = "\n".join(prompt_parts)

    try:
        raw_tags_map = gemini_run_query(
                    contents=prompt,
        )

        # Post-process to remove brands from tags
        final_tags_map = {}
        for item_in_batch in batch_data: # Iterate through the input batch_data to get brand info
            title = item_in_batch['title']
            product_brands = item_in_batch.get('brand', []) 
            
            if not isinstance(product_brands, list):
                product_brands = [] 

            # Filter out "Unknown" or "Error" brands, convert to lowercase, and remove empty strings
            valid_brands_lower = [
                b.lower().strip() for b in product_brands 
                if isinstance(b, str) and b.lower().strip() not in ["unknown", "error", ""]
            ]
            # Sort by length descending to remove longer brand names first (e.g., "google pixel" before "google")
            valid_brands_lower.sort(key=len, reverse=True)
            
            ai_generated_tags = raw_tags_map.get(title, [])
            if not isinstance(ai_generated_tags, list): # Ensure AI output for title is a list
                ai_generated_tags = []

            cleaned_product_tags = set() # Use set to avoid duplicate tags after cleaning
            for tag_str in ai_generated_tags:
                if not isinstance(tag_str, str) or not tag_str.strip(): # Skip if tag is not a string or empty
                    continue
                
                modified_tag_str = tag_str.lower() # Work with lowercase tag

                for brand_lower in valid_brands_lower:
                    if not brand_lower: # Skip empty brand strings from the sorted list
                        continue
                    # Remove brand, ensuring whole word match. re.escape handles special chars in brand names.
                    modified_tag_str = re.sub(f'\b{re.escape(brand_lower)}\b', '', modified_tag_str, flags=re.IGNORECASE)
                
                # Clean up extra spaces that might result from removal and normalize multiple spaces to one
                modified_tag_str = ' '.join(modified_tag_str.split()).strip()

                if modified_tag_str: # Add tag only if it's not empty after brand removal
                    cleaned_product_tags.add(modified_tag_str)
            
            final_tags_map[title] = sorted(list(cleaned_product_tags)) # Store unique, sorted tags

        return final_tags_map

    except Exception as e:
        print(f"Error generating tags for batch starting with '{batch_data[0]['title'] if batch_data else 'N/A'}': {e}")
        # Return a map with empty tag lists for all items in the batch in case of error
        return {item['title']: [] for item in batch_data}

# Prepare data for tag generation: unique title-description-brand sets
# Ensure 'brand' column exists from previous brand extraction cell
# We use drop_duplicates on title, assuming brand and description are dependent on title for uniqueness in this context.
df_unique_products_for_tags = df_products[['title', 'description', 'brand']].drop_duplicates(subset=['title']).reset_index(drop=True)

print("\n=== Generating Product Tags using Generative AI (brands will be excluded) ===")

# Prepare suggested tags from earlier tokenization (cell 18)
suggested_tags_from_tokenization = []
if 'category_token_counts' in globals() and isinstance(category_token_counts, dict) and 'tags' in category_token_counts:
    suggested_tags_from_tokenization = list(category_token_counts['tags'].keys())
    print(f"Using {len(suggested_tags_from_tokenization)} suggested tags from tokenization analysis.")
    # Optional: Display a sample of suggested tags
    # print(f"Sample of suggested tags: {suggested_tags_from_tokenization[:20]}")
else:
    print("Warning: 'category_token_counts' not found or 'tags' key missing. Proceeding without tag suggestions for the AI.")


batch_size_tags = 100  # Adjust based on typical length of title + description and token limits
final_merged_tags_map = {}

print(f"Processing {len(df_unique_products_for_tags)} unique title/description/brand combinations for tags in batches of {batch_size_tags}...")

for i in tqdm(range(0, len(df_unique_products_for_tags), batch_size_tags)):
    print(f"Processing tags batch {i // batch_size_tags + 1}...")
    current_batch_data = []
    for idx, row in df_unique_products_for_tags.iloc[i:i+batch_size_tags].iterrows():
        # Include brand in the data passed to the AI function for post-processing
        current_batch_data.append({'title': row['title'], 
                                   'description': row['description'], 
                                   'brand': row['brand']}) # Add brand here
    
    if not current_batch_data:
        continue
        
    # Pass suggested_tags_from_tokenization to the function
    tags_data_for_batch = generate_tags_with_ai(current_batch_data, suggested_tags_from_tokenization)
    
    if isinstance(tags_data_for_batch, dict):
        final_merged_tags_map.update(tags_data_for_batch)
    else:
        print(f"Warning: AI processing for tags batch did not return a dictionary.")
        # Default to empty list for titles in this problematic batch
        for item in current_batch_data:
            final_merged_tags_map.setdefault(item['title'], []) 
    
    time.sleep(1) # Respect API rate limits

# Assign the generated tags to the DataFrame
df_products['tags'] = df_products['title'].map(final_merged_tags_map)

# Ensure that the 'tags' column contains lists, default to empty list if not found or not a list.
df_products['tags'] = df_products['tags'].apply(
    lambda x: x if isinstance(x, list) else []
)



=== Generating Product Tags using Generative AI (brands will be excluded) ===
Using 2 suggested tags from tokenization analysis.
Processing 451 unique title/description/brand combinations for tags in batches of 100...


  0%|          | 0/5 [00:00<?, ?it/s]

Processing tags batch 1...


 20%|██        | 1/5 [00:19<01:16, 19.21s/it]

Processing tags batch 2...


...

 60%|██████    | 3/5 [00:54<00:35, 17.78s/it]

Processing tags batch 4...


 80%|████████  | 4/5 [01:09<00:16, 16.51s/it]

Processing tags batch 5...


100%|██████████| 5/5 [01:17<00:00, 15.44s/it]


In [27]:

print("\nProduct tag generation completed! (Brands excluded from tags)")
display(df_products[['title', 'tags']].head(10))


Product tag generation completed! (Brands excluded from tags)


,title,tags
0,Google Mural Tote,"[everyday use, fashionable, functional, google pride, mural design, tote bag]"
1,Google Seaport Tote,"[comfortable straps, durable, everyday essentials, google spirit, school, spacious design, tote bag, travel, work]"
2,Google Incognito Zippack V2,"[functionality, organized, privacy, secure, zippack, zippered compartments]"
3,Google Incognito Laptop Organizer V2,"[discreet protection, laptop organizer, professionals, sleek design, students, tech safe]"
4,Google Mesh Bag Blue,"[beach, breathable, everyday errands, gym, lightweight, mesh bag]"
5,#IamRemarkable Tote,"[#iamremarkable, confidence, empowering, self-promotion, tote bag]"
6,Google Campus Bike Carry Pouch,"[bike pouch, campus commuters, convenient storage, durable, easy-to-install, urban cyclists]"
7,Google Cork Pencil Pouch,"[cork pouch, durable, eco-friendly, pencil organizer, sustainable, unique]"
8,Google Campus Bike Tote Navy,"[bike rack, bike tote, commute, easy transportation, eco-friendly, navy]"
9,Google Cambridge Campus Tote,"[books, cambridge campus, durable, high-quality, laptops, stylish, tote bag]"


Lets analyze the results to see how well the model performs in identifying brands from product titles. We'll look for patterns in the extracted brand names and assess the model's accuracy.

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

print("\n=== Analyzing Product Tags ===")

# Ensure all tags are strings and convert to lowercase
df_products['tags'] = df_products['tags'].apply(
    lambda tag_list: [str(tag).lower() for tag in tag_list if isinstance(tag, str) or not pd.isna(tag)] if isinstance(tag_list, list) else []
)

# Explode the tags list into separate rows for individual tag analysis
all_tags = df_products['tags'].explode().dropna() # dropna to remove any NaN from explode if a product had an empty list initially

# Calculate tag frequencies
tag_counts = all_tags.value_counts()

print(f"Total unique tags (after lowercase): {len(tag_counts)}")
print("\nTop 20 most common tags:")
display(tag_counts.head(20))



=== Analyzing Product Tags ===
Total unique tags (after lowercase): 742

Top 20 most common tags:


tags
google           176
google pride      49
android           32
eco-friendly      29
durable           25
zip hoodie        22
sustainable       22
stylish           21
kids clothing     17
navy              15
unisex tee        15
reusable          14
black             13
grey              13
ladies tee        12
green             12
fitted tee        12
white             11
lapel pin         11
youtube           11
Name: count, dtype: int64

Let's see if we can generalize some tags to avoid 'tag swamp' and improve searchability. We'll consolidate tags by grouping similar ones together, ensuring that the final set of tags is both comprehensive and relevant.

In [29]:
from google import genai
import json
import time
from tqdm import tqdm
import pandas as pd

# Ensure client is initialized (it should be from a previous cell)
# client = genai.Client(project=project_id, location="us-central1", vertexai=True)

def consolidate_tags_with_ai(tags_batch):
    """
    Consolidate a batch of tags using Generative AI.
    """
    prompt = f"""
    You are an expert data normalizer specializing in e-commerce product tags.
    Your task is to consolidate a list of product tags to reduce variability and group semantically similar tags under a single canonical form.
    Aim to reduce the overall number of unique tags significantly, towards a target of 50-100 unique tags.

    Rules:
    1. Analyze the provided list of tags.
    2. Identify groups of tags that are synonyms, misspellings, plural/singular variations, or closely related concepts.
    3. For each group, suggest a single, concise, and representative canonical tag.
    4. If a tag is already in a good canonical form or is unique and distinct, it can map to itself.
    5. Prioritize broader terms if multiple specific terms can be grouped (e.g., "hdmi cable," "usb cable" could remain distinct or be grouped under "cables" if very aggressive consolidation is needed, but for now, try to keep meaningful distinctions unless they are very minor variations).
    6. Ensure the canonical tag is in lowercase.
    7. Remove the tags that are too generic or not useful for search (e.g., "product", "item", "thing", "location") unless they are part of a meaningful tag.
    8. Try to generate tags that are non-overlapping and distinct, avoiding redundancy.
    9. When you generate an initial list of canonical tags, ensure they are distinct and meaningful, perform another iteration of analysis based on the rules above, to avoid redundancy.
    10. Return the results as a JSON map where the key is the original tag from the input batch and the value is its suggested canonical tag.

    Example:
    Input Tags: ["running shoe", "Running Shoes", "runningshoe", "sneaker", "sporty shoe", "athletic footwear", "trainer"]
    Output JSON Map: {{
        "running shoe": "running shoe",
        "Running Shoes": "running shoe",
        "runningshoe": "running shoe",
        "runner": "running shoe",
        "white sneakers": "sneaker"
        "sneaker": "sneaker",
        "sporty shoe": "sneaker",
        "athletic footwear": "athletic shoe",
        "trainer shoes": "athletic shoe"
    }}
    (Note: This example is illustrative. Your grouping might be different based on the overall list and desired consolidation level.)

    Tags to consolidate:
    {json.dumps(tags_batch)}
    """
    try:
        consolidation_map = gemini_run_query(
            contents=prompt,
        )
        return consolidation_map
    except Exception as e:
        print(f"Error consolidating tags for batch: {e}")
        # Fallback: return a map where each tag maps to itself
        return {tag: tag for tag in tags_batch}

print("\n=== Consolidating Product Tags using Generative AI ===")

# Ensure tags are lowercase first (assuming previous cell was run)
df_products['tags'] = df_products['tags'].apply(
    lambda tag_list: [str(tag).lower() for tag in tag_list if isinstance(tag, str) or not pd.isna(tag)] if isinstance(tag_list, list) else []
)
all_lowercase_tags = df_products['tags'].explode().dropna().unique().tolist()

print(f"Found {len(all_lowercase_tags)} unique lowercase tags to consolidate.")

batch_size_consolidation = 500 # Adjust based on typical number of tags and prompt limits
master_consolidation_map = {}

if all_lowercase_tags:
    for i in tqdm(range(0, len(all_lowercase_tags), batch_size_consolidation)):
        print(f"Processing consolidation batch {i // batch_size_consolidation + 1}...")
        current_tags_batch = all_lowercase_tags[i:i+batch_size_consolidation]
        
        batch_map = consolidate_tags_with_ai(current_tags_batch)
        master_consolidation_map.update(batch_map)
        
        time.sleep(1) # Respect API rate limits
else:
    print("No tags found to consolidate.")

print(f"Master consolidation map created with {len(master_consolidation_map)} entries.")

def apply_tag_consolidation(tag_list, consolidation_map):
    if not isinstance(tag_list, list):
        return []
    consolidated_tags = set() # Use a set to store unique canonical tags for the product
    for tag in tag_list:
        tag_lower = str(tag).lower() # Ensure it's lowercase, though it should be already
        canonical_tag = consolidation_map.get(tag_lower, tag_lower) # Get from map, or use original if not found
        consolidated_tags.add(canonical_tag)
    return sorted(list(consolidated_tags))

# Apply the consolidation map to the 'tags' column
if master_consolidation_map:
    df_products['tags'] = df_products['tags'].apply(lambda x: apply_tag_consolidation(x, master_consolidation_map))
    print("\nProduct tags consolidated.")
    # Calculate and print the number of unique tags after consolidation
    all_consolidated_tags = df_products['tags'].explode().dropna().unique().tolist()
    print(f"Total unique tags after consolidation: {len(all_consolidated_tags)}")
else:
    print("\nNo consolidation map created, tags remain unchanged.")



=== Consolidating Product Tags using Generative AI ===
Found 742 unique lowercase tags to consolidate.


  0%|          | 0/2 [00:00<?, ?it/s]

Processing consolidation batch 1...


 50%|█████     | 1/2 [00:25<00:25, 25.06s/it]

Processing consolidation batch 2...


100%|██████████| 2/2 [00:38<00:00, 19.07s/it]

Master consolidation map created with 741 entries.

Product tags consolidated.
Total unique tags after consolidation: 374


In [30]:
print("number of tags before consolidation:", len(all_lowercase_tags))
print("number of tags after consolidation:", len(df_products['tags'].explode().dropna().unique().tolist()))

# Calculate and display top 20 consolidated tags with counts
consolidated_tag_counts = df_products['tags'].explode().dropna().value_counts()

# --- New: Side-by-side comparison of top 20 initial and consolidated tags ---
print("\n=== Comparison: Top 20 Initial vs. Consolidated Tags ===")

# Get top 20 initial tags (assuming 'tag_counts' is available from Cell 27)
if 'tag_counts' in globals():
    top_initial_tags = tag_counts.head(20).reset_index()
    top_initial_tags.columns = ['Initial Tag', 'Initial Count']
else:
    print("Warning: 'tag_counts' (initial tags) not found. Skipping initial tags display.")
    # Create an empty DataFrame with expected columns if tag_counts is not available
    top_initial_tags = pd.DataFrame(columns=['Initial Tag', 'Initial Count'])


top_consolidated_tags = consolidated_tag_counts.head(20).reset_index()
top_consolidated_tags.columns = ['Consolidated Tag', 'Consolidated Count']

# Combine for side-by-side display
# We'll use pd.concat. If the number of rows is different, it will fill with NaN.
# For a cleaner look, ensure both DataFrames have the same number of rows (e.g., by taking head(20) from both)
# or decide on a primary key if a merge is more appropriate (not the case here for simple top N display).

# Reset index for both to allow concatenation by position if lengths are same
# If lengths might differ and you want to align by rank:
df_comparison = pd.concat([top_initial_tags, top_consolidated_tags], axis=1)

# Fill NaN values that might appear if one list is shorter than the other (though both are head(20))
df_comparison = df_comparison.fillna('') # Fill with empty string for display

display(df_comparison)

number of tags before consolidation: 742
number of tags after consolidation: 374

=== Comparison: Top 20 Initial vs. Consolidated Tags ===


,Initial Tag,Initial Count,Consolidated Tag,Consolidated Count
0,google,176,google,190
1,google pride,49,t-shirt,97
2,android,32,google pride,49
3,eco-friendly,29,android,35
4,durable,25,eco-friendly,30
5,zip hoodie,22,water bottle,28
6,sustainable,22,sustainable,25
7,stylish,21,durable,25
8,kids clothing,17,hoodie,24
9,navy,15,stylish,24


We did pretty good, significantly reducing space of tags. Here is a sample of `tags` field after consolidation.

In [31]:
display(df_products[['title', 'tags']].head(10))

,title,tags
0,Google Mural Tote,"[everyday, functional, google pride, mural design, stylish, tote bag]"
1,Google Seaport Tote,"[comfortable, durable, everyday, google, school, spacious, tote bag, travel, work]"
2,Google Incognito Zippack V2,"[functional, organized, privacy, secure, zippered]"
3,Google Incognito Laptop Organizer V2,"[discreet, laptop, professional, sleek, student, tech]"
4,Google Mesh Bag Blue,"[beach, breathable, everyday, gym, lightweight, mesh]"
5,#IamRemarkable Tote,"[confidence, empowering, iamremarkable, self-promotion, tote bag]"
6,Google Campus Bike Carry Pouch,"[bike, campus, convenient, cyclist, durable, easy-to-install]"
7,Google Cork Pencil Pouch,"[cork, durable, eco-friendly, pencil, sustainable, unique]"
8,Google Campus Bike Tote Navy,"[bike, commuting, easy transportation, eco-friendly, navy]"
9,Google Cambridge Campus Tote,"[book, cambridge, durable, high-quality, laptop, stylish, tote bag]"


## Color Attribute Extraction
Now, for grand finale, let's extract color attributes from the product titles. We'll use the Gemini model to identify color families and specific colors associated with each product. This will help us enhance the product catalog with detailed color information. Note that we also could have analyzed the images refered in the `image` field, but for simplicity we will focus on text data only.

In [32]:
from google import genai
import json
import time
from tqdm import tqdm
import pandas as pd

# Ensure the genai client is initialized (it should be from a previous cell)
# client = genai.Client(project=project_id, location="us-central1", vertexai=True)

def generate_color_attributes_with_ai(titles_batch):
    """
    Generate colorFamilies and colors from a batch of product titles using Generative AI.
    titles_batch: A list of product titles.
    """
    prompt_parts = [
        "You are an expert in e-commerce product data analysis. Your task is to extract color families and specific colors from product information.",
        "Based on the provided product title, identify for each product:", # Changed to title only
        "1.  `colorFamilies`: A list of standard color groups.",
        "    *   Strongly recommended to use only these values: \"Red\", \"Pink\", \"Orange\", \"Yellow\", \"Purple\", \"Green\", \"Cyan\", \"Blue\", \"Brown\", \"White\", \"Gray\", \"Black\", \"Mixed\".",
        "    *   Normally, provide only 1 color family. If multiple distinct color families are clearly present and necessary, list them, but prefer \"Mixed\" if appropriate.",
        "    *   Maximum of 5 values allowed. Each value must be a UTF-8 encoded string, max 128 characters.",
        "    *   If no color is identifiable from the title, return an empty list for this key.", # Clarified source
        "2.  `colors`: A list of color display names (e.g., frontend aliases like \"scarlet red\", \"ocean blue\").",
        "    *   Normally, provide only 1 color. If multiple distinct colors are clearly present and necessary, list them.",
        "    *   Maximum of 75 colors allowed. Each value must be a UTF-8 encoded string, max 128 characters.",
        "    *   If no color is identifiable from the title, return an empty list for this key.", # Clarified source
        "",
        "Return results as a single JSON map where the key is the original product title, and the value is a JSON object with two keys: \"colorFamilies\" (a list of strings) and \"colors\" (a list of strings).",
        "Example for a single product entry in the map:",
        "\"Product Title Example\": { \"colorFamilies\": [\"Red\"], \"colors\": [\"Scarlet Red\", \"Crimson\"] }",
        "If no colors can be determined for a product from its title, its entry should be like:", # Clarified source
        "\"Another Product Title\": { \"colorFamilies\": [], \"colors\": [] }",
        "Ensure the entire output is a valid JSON object (a single map).",
        "\nProduct titles to analyze:" # Changed from "Product data to analyze:"
    ]

    for i, title_str in enumerate(titles_batch):
        prompt_parts.append(f"{i+1}. Title: {title_str}") # Use title_str directly
        
    prompt = "\n".join(prompt_parts)

    try:
        color_attributes_map = gemini_run_query(
            contents=prompt,
        )
        return color_attributes_map
    except Exception as e:
        print(f"Error generating color attributes for batch starting with title '{titles_batch[0] if titles_batch else 'N/A'}': {e}")
        # Return a map with empty color lists for all items in the batch in case of error
        return {title: {"colorFamilies": [], "colors": []} for title in titles_batch}

# Prepare data for color attribute generation
# We use drop_duplicates on title.
# Ensure 'description' column might still be needed for display, but not for AI processing here.
# If 'description' is not in df_products for other reasons, ensure it's handled or remove from display.
if 'description' not in df_products.columns:
    df_products['description'] = "Not available" 

df_unique_titles_for_colors = df_products[['title']].drop_duplicates(subset=['title']).reset_index(drop=True)

print("\n=== Generating Color Attributes (Families and Colors) using Generative AI (from Titles only) ===")
batch_size_colors = 200  # Adjust based on typical length of title and token limits
final_merged_color_attributes_map = {}

print(f"Processing {len(df_unique_titles_for_colors)} unique titles for color attributes in batches of {batch_size_colors}...")

for i in tqdm(range(0, len(df_unique_titles_for_colors), batch_size_colors)):
    print(f"Processing color attributes batch {i // batch_size_colors + 1}...")
    current_batch_titles_list = []
    for idx, row in df_unique_titles_for_colors.iloc[i:i+batch_size_colors].iterrows():
        current_batch_titles_list.append(row['title']) # Append only title string
    
    if not current_batch_titles_list:
        continue
        
    batch_color_map = generate_color_attributes_with_ai(current_batch_titles_list)
    
    if isinstance(batch_color_map, dict):
        final_merged_color_attributes_map.update(batch_color_map)
    else:
        print(f"Warning: AI processing for color attributes batch did not return a dictionary for titles starting with '{current_batch_titles_list[0] if current_batch_titles_list else 'N/A'}'.")
        for title_str in current_batch_titles_list:
            final_merged_color_attributes_map.setdefault(title_str, {"colorFamilies": [], "colors": []})
    
    time.sleep(1) # Respect API rate limits

# Assign the generated color attributes to the DataFrame
def get_color_families_from_map(title):
    return final_merged_color_attributes_map.get(title, {}).get('colorFamilies', [])

def get_colors_from_map(title):
    return final_merged_color_attributes_map.get(title, {}).get('colors', [])

df_products['colorFamilies'] = df_products['title'].apply(get_color_families_from_map)
df_products['colors'] = df_products['title'].apply(get_colors_from_map)

# Ensure the new columns are lists
df_products['colorFamilies'] = df_products['colorFamilies'].apply(lambda x: x if isinstance(x, list) else [])
df_products['colors'] = df_products['colors'].apply(lambda x: x if isinstance(x, list) else [])



=== Generating Color Attributes (Families and Colors) using Generative AI (from Titles only) ===
Processing 451 unique titles for color attributes in batches of 200...


  0%|          | 0/3 [00:00<?, ?it/s]

Processing color attributes batch 1...


 33%|███▎      | 1/3 [00:32<01:05, 32.55s/it]

Processing color attributes batch 2...


 67%|██████▋   | 2/3 [00:59<00:29, 29.44s/it]

Processing color attributes batch 3...


100%|██████████| 3/3 [01:08<00:00, 22.77s/it]


In [33]:

print("\nColor attribute generation completed! (from Titles only)")
display(
    df_products[
        (df_products['colorFamilies'].apply(lambda x: isinstance(x, list) and len(x) > 0)) |
        (df_products['colors'].apply(lambda x: isinstance(x, list) and len(x) > 0))
    ][['title', 'colorFamilies', 'colors']].head(10)
)


Color attribute generation completed! (from Titles only)


,title,colorFamilies,colors
4,Google Mesh Bag Blue,[Blue],[Blue]
7,Google Cork Pencil Pouch,[Brown],[Cork]
8,Google Campus Bike Tote Navy,[Blue],[Navy]
16,Google Confetti Tote White,[White],[White]
17,Google Large Tote White,[White],[White]
18,Google Hemp Tote,[Brown],[Hemp]
23,Google Flat Front Bag Grey,[Gray],[Grey]
24,Google Cork Tablet Case,[Brown],[Cork]
32,Android Flex Journal White,[White],[White]
35,Google ApPeel Journal Red,[Red],[Red]


## Create and Populate Enhanced Product Table

Let's take a look at the df_products dataframe now, that we have enriched our product data with additional attributes such as brand, description, tags, and color information. It contains the original product data along with the enriched fields we have generated using Generative AI models.

In [34]:
display(df_products.head(50))

,title,pattern,size,audience,brand,description,tags,colorFamilies,colors
0,Google Mural Tote,None,None,None,[Google],"Carry your essentials in style with the Google Mural Tote. Featuring a unique mural design, this tote is both fashionable and functional. Perfect for everyday use, it's a great way to show off your Google pride.","[everyday, functional, google pride, mural design, stylish, tote bag]",[],[]
1,Google Seaport Tote,None,None,None,[Google],"Show your Google spirit with the Google Seaport Tote. This stylish and durable tote bag is perfect for carrying your everyday essentials. Its spacious design and comfortable straps make it ideal for work, school, or travel.","[comfortable, durable, everyday, google, school, spacious, tote bag, travel, work]",[],[]
2,Google Incognito Zippack V2,None,None,None,[Google],Keep your belongings secure and organized with the Google Incognito Zippack V2. This updated design offers enhanced security with its zippered compartments. Ideal for those who value privacy and functionality.,"[functional, organized, privacy, secure, zippered]",[],[]
3,Google Incognito Laptop Organizer V2,None,None,None,[Google],The Google Incognito Laptop Organizer V2 offers discreet protection and organization for your laptop and accessories. This updated version provides enhanced features and a sleek design. Perfect for professionals and students alike who need to keep their tech safe.,"[discreet, laptop, professional, sleek, student, tech]",[],[]
4,Google Mesh Bag Blue,None,None,None,[Google],"The Google Mesh Bag in Blue is a lightweight and breathable option for carrying your gear. Perfect for the gym, beach, or everyday errands. Its mesh design allows for ventilation and easy visibility of contents.","[beach, breathable, everyday, gym, lightweight, mesh]",[Blue],[Blue]
5,#IamRemarkable Tote,None,None,None,[Unknown],Celebrate the #IamRemarkable initiative with this empowering tote bag. Perfect for carrying your daily essentials while promoting self-promotion and confidence. Show your support and spread the message wherever you go.,"[confidence, empowering, iamremarkable, self-promotion, tote bag]",[],[]
6,Google Campus Bike Carry Pouch,None,None,None,[Google],The Google Campus Bike Carry Pouch is designed for convenient storage while cycling. Securely carry your essentials on your bike with this durable and easy-to-install pouch. Perfect for campus commuters and urban cyclists.,"[bike, campus, convenient, cyclist, durable, easy-to-install]",[],[]
7,Google Cork Pencil Pouch,None,None,None,[Google],"Store your writing essentials in sustainable style with the Google Cork Pencil Pouch. Made from eco-friendly cork, this pouch is both durable and unique. Keep your pens and pencils organized in a responsible way.","[cork, durable, eco-friendly, pencil, sustainable, unique]",[Brown],[Cork]
8,Google Campus Bike Tote Navy,None,None,None,[Google],"The Google Campus Bike Tote in Navy is designed for easy and stylish transportation on your bike. This tote securely attaches to your bike rack, making it perfect for carrying groceries, books, or other essentials. Enjoy a convenient and eco-friendly way to commute.","[bike, commuting, easy transportation, eco-friendly, navy]",[Blue],[Navy]
9,Google Cambridge Campus Tote,None,None,None,"[Google, cambridge]","Show your Cambridge campus pride with this durable and stylish tote. Perfect for carrying books, laptops, and other essentials around campus. Made with high-quality materials for long-lasting use.","[book, cambridge, durable, high-quality, laptop, stylish, tote bag]",[],[]



We will now create a new table in BigQuery to store this enhanced product data. This table will be named `products_enhanced` and will include all the original fields along with the newly generated attributes..

The process involves the following key stages to create and populate the `products_enhanced` table in BigQuery:

1.  **Table Cloning for Baseline:**
    *   A new table, `products_enhanced`, is created by cloning the existing `retail.products` table. This ensures that all original data and the existing schema structure are preserved as a baseline.

2.  **Schema Extension for Enriched Attributes:**
    *   The schema of the newly cloned `products_enhanced` table is extended. This involves adding new columns required to store the enriched product attributes (such as brands, description, tags, and color information) that are not present in the original table. The complete target schema, inclusive of these new fields, is typically predefined (e.g., loaded from a GCS file).

3.  **Populating Extended Attributes:**
    *   The newly added columns in the `products_enhanced` table are populated with the enriched data generated within this notebook. This is achieved by:
        *   Preparing the enriched data (e.g., from the `df_products` Pandas DataFrame).
        *   Loading this prepared data into a temporary BigQuery table.
        *   Executing a `MERGE` operation to update the `products_enhanced` table, matching records by product ID and filling in the values for the extended attributes.

4.  **Verification and Cleanup:**
    *   Post-population, a verification step is performed by querying a sample of the updated records to ensure data integrity.
    *   Any temporary resources, such as the intermediate table used for the merge, are cleaned up.

### Table cloning and schema extension

In [35]:
from google.cloud import bigquery
from google.cloud import storage
import json

# Ensure BigQuery client is initialized
client = bigquery.Client(project=project_id)

# Define table names and schema path
TABLE_ENHANCED_ID = f"{project_id}.retail.products_enhanced"
SOURCE_TABLE_ID = f"{project_id}.retail.products"
SCHEMA_GCS_PATH = f"gs://{SCRIPTS_BUCKET}/vaisc-configs/retail_products_schema.json"

print(f"Target table for creation/update: {TABLE_ENHANCED_ID}")
print(f"Source table for copy: {SOURCE_TABLE_ID}")
print(f"Schema GCS path for adding columns: {SCHEMA_GCS_PATH}")

def bq_schema_field_to_ddl_type(field: bigquery.SchemaField) -> str:
    """Converts a BigQuery SchemaField to its DDL type string."""
    # Base type determination
    if field.field_type == "RECORD": # BigQuery client uses "RECORD" for STRUCTs
        if not field.fields: # Handle empty STRUCT if it can occur
            return "STRUCT<>"
        subfields_ddl = ", ".join([f"`{sf.name}` {bq_schema_field_to_ddl_type(sf)}" for sf in field.fields])
        base_ddl_type = f"STRUCT<{subfields_ddl}>"
    else: # Simple types like STRING, INTEGER, FLOAT64, BOOLEAN, TIMESTAMP, DATE etc.
        base_ddl_type = field.field_type

    # Apply ARRAY if mode is REPEATED
    if field.mode == "REPEATED":
        return f"ARRAY<{base_ddl_type}>"
    else:
        return base_ddl_type

try:
    # 1. Delete the enhanced table if it exists
    client.delete_table(TABLE_ENHANCED_ID, not_found_ok=True)
    print(f"Deleted table {TABLE_ENHANCED_ID} if it existed.")

    # 2. Create the enhanced table as a copy of the source table
    # create_as_copy_query = f"CREATE TABLE `{TABLE_ENHANCED_ID}` AS SELECT * FROM `{SOURCE_TABLE_ID}`"
    print(f"Executing: copying {SOURCE_TABLE_ID} to {TABLE_ENHANCED_ID}...")
    # copy_job = client.query(create_as_copy_query)
    copy_job = client.copy_table(SOURCE_TABLE_ID, TABLE_ENHANCED_ID)
    copy_job.result()  # Wait for the job to complete
    print(f"Successfully created {TABLE_ENHANCED_ID} as a copy of {SOURCE_TABLE_ID}.")

    # 3. Load target schema from GCS
    gcs_schema_fields = []
    try:
        storage_client = storage.Client(project=project_id)
        bucket_name, blob_name = SCHEMA_GCS_PATH.replace("gs://", "").split("/", 1)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        schema_json_string = blob.download_as_string()
        schema_list_of_dicts = json.loads(schema_json_string)
        gcs_schema_fields = [bigquery.SchemaField.from_api_repr(field_dict) for field_dict in schema_list_of_dicts]
        print(f"Successfully loaded target schema with {len(gcs_schema_fields)} fields from {SCHEMA_GCS_PATH}")
    except Exception as e:
        print(f"Error loading schema from GCS: {e}. Cannot proceed with adding columns.")
        raise # Re-raise to stop execution if schema loading fails

    if not gcs_schema_fields:
        print("GCS Schema was not loaded or is empty, cannot proceed with adding columns.")
    else:
               # 4. Get current schema of the newly created table
        current_table = client.get_table(TABLE_ENHANCED_ID)
        current_field_names = {field.name for field in current_table.schema}
        print(f"Current fields in {TABLE_ENHANCED_ID} after copy: {current_field_names}")

        # 5. Identify and add missing columns
        added_columns_count = 0
        alter_statements = []
        for gcs_field in gcs_schema_fields:
            if gcs_field.name not in current_field_names:
                column_name = gcs_field.name
                column_type_ddl = bq_schema_field_to_ddl_type(gcs_field)
                
                alter_sql = f"ALTER TABLE `{TABLE_ENHANCED_ID}` ADD COLUMN `{column_name}` {column_type_ddl}"
                alter_statements.append(alter_sql)
                print(f"  Will add column: `{column_name}` with type {column_type_ddl}")

        if alter_statements:
            print(f"\nApplying {len(alter_statements)} schema alterations...")
            for i, alter_sql in enumerate(alter_statements):
                try:
                    print(f"Executing ALTER statement {i+1}/{len(alter_statements)}: {alter_sql}")
                    alter_job = client.query(alter_sql)
                    alter_job.result()  # Wait for completion
                    print(f"Successfully executed: {alter_sql}")
                    added_columns_count += 1
                except Exception as e:
                    print(f"Error executing ALTER statement '{alter_sql}': {e}")
                    # Optionally, decide if to stop or continue. For now, it continues.
            print(f"Finished applying schema alterations. {added_columns_count} columns potentially added.")
        else:
            print("No new columns to add. Schema of copied table already contains all columns defined in GCS schema (by name).")

        # Verify final schema (optional, but good for debugging)
        final_table = client.get_table(TABLE_ENHANCED_ID)
        final_field_names = {field.name for field in final_table.schema}
        print(f"Final fields in {TABLE_ENHANCED_ID}: {final_field_names}")
        # print("Final schema details:")
        # for field in final_table.schema:
        #     print(f"  - {field.name}: {bq_schema_field_to_ddl_type(field)} (Mode: {field.mode})")

except Exception as e:
    print(f"An error occurred during table creation or schema update: {e}")



Target table for creation/update: artilekt-vaisc-csb.retail.products_enhanced
Source table for copy: artilekt-vaisc-csb.retail.products
Schema GCS path for adding columns: gs://artilekt-vaisc-csb_scripts/vaisc-configs/retail_products_schema.json
Deleted table artilekt-vaisc-csb.retail.products_enhanced if it existed.
Executing: copying artilekt-vaisc-csb.retail.products to artilekt-vaisc-csb.retail.products_enhanced...
Deleted table artilekt-vaisc-csb.retail.products_enhanced if it existed.
Executing: copying artilekt-vaisc-csb.retail.products to artilekt-vaisc-csb.retail.products_enhanced...
Successfully created artilekt-vaisc-csb.retail.products_enhanced as a copy of artilekt-vaisc-csb.retail.products.
Successfully created artilekt-vaisc-csb.retail.products_enhanced as a copy of artilekt-vaisc-csb.retail.products.
Successfully loaded target schema with 31 fields from gs://artilekt-vaisc-csb_scripts/vaisc-configs/retail_products_schema.json
Current fields in artilekt-vaisc-csb.retail.

### Data population with extended attributes

In [36]:

# Display the column names of the df_products DataFrame
print(df_products.columns)
display(df_products.head(100))

Index(['title', 'pattern', 'size', 'audience', 'brand', 'description', 'tags',
       'colorFamilies', 'colors'],
      dtype='object')


,title,pattern,size,audience,brand,description,tags,colorFamilies,colors
0,Google Mural Tote,None,None,None,[Google],"Carry your essentials in style with the Google Mural Tote. Featuring a unique mural design, this tote is both fashionable and functional. Perfect for everyday use, it's a great way to show off your Google pride.","[everyday, functional, google pride, mural design, stylish, tote bag]",[],[]
1,Google Seaport Tote,None,None,None,[Google],"Show your Google spirit with the Google Seaport Tote. This stylish and durable tote bag is perfect for carrying your everyday essentials. Its spacious design and comfortable straps make it ideal for work, school, or travel.","[comfortable, durable, everyday, google, school, spacious, tote bag, travel, work]",[],[]
2,Google Incognito Zippack V2,None,None,None,[Google],Keep your belongings secure and organized with the Google Incognito Zippack V2. This updated design offers enhanced security with its zippered compartments. Ideal for those who value privacy and functionality.,"[functional, organized, privacy, secure, zippered]",[],[]
3,Google Incognito Laptop Organizer V2,None,None,None,[Google],The Google Incognito Laptop Organizer V2 offers discreet protection and organization for your laptop and accessories. This updated version provides enhanced features and a sleek design. Perfect for professionals and students alike who need to keep their tech safe.,"[discreet, laptop, professional, sleek, student, tech]",[],[]
4,Google Mesh Bag Blue,None,None,None,[Google],"The Google Mesh Bag in Blue is a lightweight and breathable option for carrying your gear. Perfect for the gym, beach, or everyday errands. Its mesh design allows for ventilation and easy visibility of contents.","[beach, breathable, everyday, gym, lightweight, mesh]",[Blue],[Blue]
5,#IamRemarkable Tote,None,None,None,[Unknown],Celebrate the #IamRemarkable initiative with this empowering tote bag. Perfect for carrying your daily essentials while promoting self-promotion and confidence. Show your support and spread the message wherever you go.,"[confidence, empowering, iamremarkable, self-promotion, tote bag]",[],[]
6,Google Campus Bike Carry Pouch,None,None,None,[Google],The Google Campus Bike Carry Pouch is designed for convenient storage while cycling. Securely carry your essentials on your bike with this durable and easy-to-install pouch. Perfect for campus commuters and urban cyclists.,"[bike, campus, convenient, cyclist, durable, easy-to-install]",[],[]
7,Google Cork Pencil Pouch,None,None,None,[Google],"Store your writing essentials in sustainable style with the Google Cork Pencil Pouch. Made from eco-friendly cork, this pouch is both durable and unique. Keep your pens and pencils organized in a responsible way.","[cork, durable, eco-friendly, pencil, sustainable, unique]",[Brown],[Cork]
8,Google Campus Bike Tote Navy,None,None,None,[Google],"The Google Campus Bike Tote in Navy is designed for easy and stylish transportation on your bike. This tote securely attaches to your bike rack, making it perfect for carrying groceries, books, or other essentials. Enjoy a convenient and eco-friendly way to commute.","[bike, commuting, easy transportation, eco-friendly, navy]",[Blue],[Navy]
9,Google Cambridge Campus Tote,None,None,None,"[Google, cambridge]","Show your Cambridge campus pride with this durable and stylish tote. Perfect for carrying books, laptops, and other essentials around campus. Made with high-quality materials for long-lasting use.","[book, cambridge, durable, high-quality, laptop, stylish, tote bag]",[],[]


In [37]:
import pandas as pd
from google.cloud import bigquery

# Ensure BigQuery client is initialized (it should be from previous cells)
client = bigquery.Client(project=project_id)

# 1. Prepare DataFrame for BigQuery load
# Select relevant columns and rename to match BQ schema
df_update = df_products[['title', 'description', 'brand', 'tags', 'colorFamilies', 'colors', 'size', 'pattern', 'audience']].copy()
df_update.rename(columns={
    'brand': 'brands',
    'pattern': 'patterns',
    'size': 'sizes'
}, inplace=True)

# Ensure list columns are not NaN, replace with empty list if so
# Convert comma-separated string columns ('patterns', 'sizes') to lists
def string_to_list(s):
    if pd.isna(s) or s == '':
        return []
    return [item.strip() for item in s.split(',')]

df_update['patterns'] = df_update['patterns'].apply(string_to_list)
df_update['sizes'] = df_update['sizes'].apply(string_to_list)

# Ensure other list columns are indeed lists and handle NaNs
for col in ['brands', 'tags', 'colorFamilies', 'colors']:
    df_update[col] = df_update[col].apply(lambda x: x if isinstance(x, list) else [])

# Ensure 'audience' is a dict, replace None/NaN with default dict structure for BQ STRUCT
def ensure_audience_structure(aud):
    if pd.isna(aud) or not isinstance(aud, dict):
        return {"genders": [], "ageGroups": []}
    # Ensure keys exist
    if 'genders' not in aud or not isinstance(aud['genders'], list):
        aud['genders'] = []
    if 'ageGroups' not in aud or not isinstance(aud['ageGroups'], list):
        aud['ageGroups'] = []
    return aud

df_update['audience'] = df_update['audience'].apply(ensure_audience_structure)


print(f"Prepared DataFrame with {len(df_update)} records for update.")
display(df_update)

# Define temporary table ID for loading data
TEMP_TABLE_ID = f"{project_id}.retail.products_temp_update"

# 2. Load DataFrame to a temporary BigQuery table
try:
    print(f"Loading data into temporary table: {TEMP_TABLE_ID}...")
    # Delete temp table if it exists from a previous run
    client.delete_table(TEMP_TABLE_ID, not_found_ok=True)
    print(f"Deleted temporary table {TEMP_TABLE_ID} if it existed.")

    job_config = bigquery.LoadJobConfig(
        autodetect=True # BQ can infer schema for DataFrames, including dicts as STRUCTs
    )
    load_job = client.load_table_from_dataframe(
        df_update, TEMP_TABLE_ID, job_config=job_config
    )
    load_job.result()  # Wait for the job to complete
    print(f"Successfully loaded {load_job.output_rows} rows into {TEMP_TABLE_ID}.")

    # 3. Execute MERGE statement to update the target table
    merge_query = f"""
    MERGE `{TABLE_ENHANCED_ID}` T
    USING `{TEMP_TABLE_ID}` S
    ON T.title = S.title
    WHEN MATCHED THEN
        UPDATE SET
            T.description = S.description,
            T.brands = S.brands,
            T.tags = S.tags,
            T.colorInfo = STRUCT(
                S.colorFamilies AS colorFamilies, 
                S.colors AS colors
            ),
            T.patterns = S.patterns,
            T.sizes = S.sizes,
            T.audience = STRUCT(
                S.audience.genders AS genders,
                S.audience.ageGroups AS ageGroups
            )
    """
    print("\nExecuting MERGE statement...")
    print(merge_query)
    merge_job = client.query(merge_query)
    merge_job.result()  # Wait for the job to complete
    print(f"MERGE statement completed. {merge_job.num_dml_affected_rows} rows affected in {TABLE_ENHANCED_ID}.")

finally:
    # 4. Clean up the temporary table
    client.delete_table(TEMP_TABLE_ID, not_found_ok=True)
    print(f"\nTemporary table {TEMP_TABLE_ID} deleted.")


Prepared DataFrame with 451 records for update.


,title,description,brands,tags,colorFamilies,colors,sizes,patterns,audience
0,Google Mural Tote,"Carry your essentials in style with the Google Mural Tote. Featuring a unique mural design, this tote is both fashionable and functional. Perfect for everyday use, it's a great way to show off your Google pride.",[Google],"[everyday, functional, google pride, mural design, stylish, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
1,Google Seaport Tote,"Show your Google spirit with the Google Seaport Tote. This stylish and durable tote bag is perfect for carrying your everyday essentials. Its spacious design and comfortable straps make it ideal for work, school, or travel.",[Google],"[comfortable, durable, everyday, google, school, spacious, tote bag, travel, work]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
2,Google Incognito Zippack V2,Keep your belongings secure and organized with the Google Incognito Zippack V2. This updated design offers enhanced security with its zippered compartments. Ideal for those who value privacy and functionality.,[Google],"[functional, organized, privacy, secure, zippered]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
3,Google Incognito Laptop Organizer V2,The Google Incognito Laptop Organizer V2 offers discreet protection and organization for your laptop and accessories. This updated version provides enhanced features and a sleek design. Perfect for professionals and students alike who need to keep their tech safe.,[Google],"[discreet, laptop, professional, sleek, student, tech]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
4,Google Mesh Bag Blue,"The Google Mesh Bag in Blue is a lightweight and breathable option for carrying your gear. Perfect for the gym, beach, or everyday errands. Its mesh design allows for ventilation and easy visibility of contents.",[Google],"[beach, breathable, everyday, gym, lightweight, mesh]",[Blue],[Blue],[],[],"{'genders': [], 'ageGroups': []}"
5,#IamRemarkable Tote,Celebrate the #IamRemarkable initiative with this empowering tote bag. Perfect for carrying your daily essentials while promoting self-promotion and confidence. Show your support and spread the message wherever you go.,[Unknown],"[confidence, empowering, iamremarkable, self-promotion, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
6,Google Campus Bike Carry Pouch,The Google Campus Bike Carry Pouch is designed for convenient storage while cycling. Securely carry your essentials on your bike with this durable and easy-to-install pouch. Perfect for campus commuters and urban cyclists.,[Google],"[bike, campus, convenient, cyclist, durable, easy-to-install]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
7,Google Cork Pencil Pouch,"Store your writing essentials in sustainable style with the Google Cork Pencil Pouch. Made from eco-friendly cork, this pouch is both durable and unique. Keep your pens and pencils organized in a responsible way.",[Google],"[cork, durable, eco-friendly, pencil, sustainable, unique]",[Brown],[Cork],[],[],"{'genders': [], 'ageGroups': []}"
8,Google Campus Bike Tote Navy,"The Google Campus Bike Tote in Navy is designed for easy and stylish transportation on your bike. This tote securely attaches to your bike rack, making it perfect for carrying groceries, books, or other essentials. Enjoy a convenient and eco-friendly way to commute.",[Google],"[bike, commuting, easy transportation, eco-friendly, navy]",[Blue],[Navy],[],[],"{'genders': [], 'ageGroups': []}"
9,Google Cambridge Campus Tote,"Show your Cambridge campus pride with this durable and stylish tote. Perfect for carrying books, laptops, and other essentials around campus. Made with high-quality materials for long-lasting use.","[Google, cambridge]","[book, cambridge, durable, high-quality, laptop, stylish, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"


Loading data into temporary table: artilekt-vaisc-csb.retail.products_temp_update...
Deleted temporary table artilekt-vaisc-csb.retail.products_temp_update if it existed.
Deleted temporary table artilekt-vaisc-csb.retail.products_temp_update if it existed.
Successfully loaded 451 rows into artilekt-vaisc-csb.retail.products_temp_update.

Executing MERGE statement...

    MERGE `artilekt-vaisc-csb.retail.products_enhanced` T
    USING `artilekt-vaisc-csb.retail.products_temp_update` S
    ON T.title = S.title
    WHEN MATCHED THEN
        UPDATE SET
            T.description = S.description,
            T.brands = S.brands,
            T.tags = S.tags,
            T.colorInfo = STRUCT(
                S.colorFamilies AS colorFamilies, 
                S.colors AS colors
            ),
            T.patterns = S.patterns,
            T.sizes = S.sizes,
            T.audience = STRUCT(
                S.audience.genders AS genders,
                S.audience.ageGroups AS ageGroups
       

In [38]:

# 5. Verification Query
print("\nFetching a sample of updated records from products_enhanced...")
verification_query = f"""
SELECT 
    id, 
    description, 
    brands, 
    tags,
    colorInfo.colorFamilies AS colorFamilies,
    colorInfo.colors AS colors,
    patterns,
    sizes,
    audience.genders AS audience_genders,
    audience.ageGroups AS audience_ageGroups
FROM `{TABLE_ENHANCED_ID}`
WHERE description IS NOT NULL 
    OR ARRAY_LENGTH(brands) > 0 
    OR ARRAY_LENGTH(tags) > 0 
    OR ARRAY_LENGTH(colorInfo.colorFamilies) > 0 
    OR ARRAY_LENGTH(colorInfo.colors) > 0
    OR ARRAY_LENGTH(patterns) > 0
    OR ARRAY_LENGTH(sizes) > 0
    OR ARRAY_LENGTH(audience.genders) > 0
    OR ARRAY_LENGTH(audience.ageGroups) > 0
LIMIT 100
"""
df_verified = client.query(verification_query).to_dataframe()
display(df_verified)


Fetching a sample of updated records from products_enhanced...


,id,description,brands,tags,colorFamilies,colors,patterns,sizes,audience_genders,audience_ageGroups
0,GGOEGKBA168099,Unleash your creativity and problem-solving skills with the Ball of Whacks. This magnetic puzzle is perfect for sparking innovation and stimulating your mind.,[Unknown],"[ball of whacks, creative, problem-solving, puzzle]",[],[],[],[],[],[]
1,GGOEGABQ170299,Add a touch of nostalgia to your collection with the Google Chrome Dino Dark Mode Collectible. A fun and unique piece for any Google enthusiast.,[Google],"[collectible, dark mode, dino, google, nostalgia]",[Black],[Dark],[],[],[],[]
2,GGCOGCBA164499,"Stay cozy and warm with the Google Knit Blanket. Perfect for snuggling up on the couch, this blanket features a comfortable knit design and subtle Google branding.",[google],"[blanket, branded, cozy, google]",[],[],[],[],[],[]
3,GGCOGOAC101259,"Show your support for the #IamRemarkable initiative with this stylish pen. Perfect for everyday use, this pen serves as a reminder to celebrate your accomplishments. Write with confidence and inspire others to do the same.",[Unknown],"[celebrate, iamremarkable, pen, stylish]",[],[],[],[],[],[]
4,GGOEGCKA126299,"Show your support for the LGBTQ+ community with this Google Pride Sticker. Perfect for laptops, water bottles, and more.",[Google],"[google, lgbtq+, pride, support]",[Mixed],[Pride],[],[],[],[]
5,GGOEGGOA017399,"The Google Maze Pen is a fun and functional writing tool. Featuring a unique maze design on the barrel, this pen adds a playful touch to your everyday writing. Enjoy smooth writing and a creative design with the Google Maze Pen.",[Google],"[fun, functional, maze, smooth]",[],[],[],[],[],[]
6,GGOEAFKQ130599,"Show your Android pride with the Iconic 4in Decal. Perfect for laptops, cars, and more, this decal is a great way to display your love for Android.",[Android],[decal],[],[],[],[],[],[]
7,GGOEGFKA022299,Google Keyboard DOT Sticker,[Google],[sticker],[],[],[DOT],[],[],[]
8,GGOEGDWC166199,Enjoy your favorite hot beverages in style with this Google Blue Stoneware Mug. A durable and elegant addition to your mug collection.,[Google],"[blue, durable, elegant, google, mug]",[Blue],[Blue],[],[],[],[]
9,GGOECCKQ167499,"Showcase your support for Google Cloud with the Google Cloud Sticker. This sticker is perfect for laptops, water bottles, or any surface where you want to display your cloud allegiance.",[google],"[decal, google cloud, sticker]",[],[],[],[],[],[]


# Verification of Data enriched catalog
Now that we have populated the `products_enhanced` table with enriched product data, it's crucial to verify the integrity and accuracy of the enriched attributes. This step ensures that the data meets our expectations and is ready for use in search and recommendation systems.
We will use the Vertex AI Search for Commerce validation tool to perform this verification. The validation tool allows us to check the consistency and correctness of the enriched data against the expected schema and values.

## Data loading
Let's load the enriched product data from the `products_enhanced` table into the Vertex AI Search for Commerce catalog. For testing purposes, we will use Branch 2 of the catalog. This would allow us to validate the enriched data without affecting the production catalog.

We will use python script to load the data. You can also use the [Vertex AI Search for Commerce console](https://console.cloud.google.com/ai/retail/catalogs/default_catalog/data/catalog) to load the data, but using python script allows us to automate the process and easily repeat it in the future.

In [39]:
from google.cloud import retail_v2
from google.cloud.retail_v2.types import ImportProductsRequest, ProductInputConfig, BigQuerySource, ImportErrorsConfig
from google.cloud import storage
import json
import time

# Define Branch 2 for testing enriched data
BRANCH_2 = f"projects/{project_id}/locations/global/catalogs/default_catalog/branches/2"
ENHANCED_TABLE_FULL_ID = f"{project_id}.retail.products_enhanced"

print(f"Loading enriched data from {ENHANCED_TABLE_FULL_ID} into Branch 2: {BRANCH_2}")

def load_products_to_branch_from_bq(branch_name, table_id, reconciliation_mode="FULL"):
    """
    Load products from BigQuery table into a specific catalog branch.
    
    Args:
        branch_name: Target branch (e.g., "projects/.../branches/2")
        table_id: BigQuery table ID (e.g., "products_enhanced")
        reconciliation_mode: "INCREMENTAL" or "FULL"
    """
    # Initialize the ProductService client
    client = retail_v2.ProductServiceClient()

    # Parse dataset_id and table_id from the full table_id string if needed
    # If table_id is in the form "project.dataset.table", split it
    if "." in table_id:
        parts = table_id.split(".")
        if len(parts) == 3:
            _project_id, dataset_id, table_id_short = parts
        elif len(parts) == 2:
            dataset_id, table_id_short = parts
        else:
            dataset_id = "retail"
            table_id_short = table_id
    else:
        dataset_id = "retail"
        table_id_short = table_id

    bq_source = BigQuerySource(
        project_id=project_id,
        dataset_id=dataset_id,
        table_id=table_id_short,
        # gcs_staging_dir=f"gs://{SCRIPTS_BUCKET}/retail_import_staging/",
        data_schema="product"
    )

    input_config = ProductInputConfig(
        big_query_source=bq_source
    )

    errors_config = ImportErrorsConfig(
        gcs_prefix=f"gs://{SCRIPTS_BUCKET}/retail_import_errors/"
    )

    request = ImportProductsRequest(
        parent=branch_name,
        input_config=input_config,
        errors_config=errors_config,
        reconciliation_mode=reconciliation_mode,
        notification_pubsub_topic=None
    )

    try:
        print(f"Starting import operation for branch: {branch_name}")
        print(f"Source table: {table_id}")
        print(f"Reconciliation mode: {reconciliation_mode}")
        operation = client.import_products(request=request)
        print(f"Import operation started. Operation name: {operation.operation.name}")
        print("Waiting for import operation to complete...")
        result = operation.result(timeout=1800)
        print("Import operation completed successfully!")
        print(f"Operation result: {result}")
        return operation.operation.name, result
    except Exception as e:
        print(f"Error during import operation: {e}")
        return None, None

def check_import_operation_status(operation_name):
    """
    Check the status of an import operation.
    """
    try:
        from google.cloud import retail_v2
        client = retail_v2.ProductServiceClient()
        operation = client.transport.operations_client.get_operation(
            name=operation_name
        )
        print(f"Operation status: {operation}")
        return operation
    except Exception as e:
        print(f"Error checking operation status: {e}")
        return None

# Execute the data loading
print("=== Loading Enhanced Product Data to Branch 2 ===")

# Load data with INCREMENTAL mode (you can change to "FULL" if needed)
operation_name, result = load_products_to_branch_from_bq(
    branch_name=BRANCH_2,
    table_id="products_enhanced",
    reconciliation_mode="INCREMENTAL"
)

if operation_name:
    print(f"\nImport operation completed successfully!")
    print(f"Operation name: {operation_name}")
    if result:
        print(f"Import result: {result}")
else:
    print("Import operation failed. Please check the error messages above.")

# Optional: Check operation status after completion
if operation_name:
    print("\n=== Checking Final Operation Status ===")
    final_status = check_import_operation_status(operation_name)

Loading enriched data from artilekt-vaisc-csb.retail.products_enhanced into Branch 2: projects/artilekt-vaisc-csb/locations/global/catalogs/default_catalog/branches/2
=== Loading Enhanced Product Data to Branch 2 ===
Starting import operation for branch: projects/artilekt-vaisc-csb/locations/global/catalogs/default_catalog/branches/2
Source table: products_enhanced
Reconciliation mode: INCREMENTAL
Starting import operation for branch: projects/artilekt-vaisc-csb/locations/global/catalogs/default_catalog/branches/2
Source table: products_enhanced
Reconciliation mode: INCREMENTAL
Import operation started. Operation name: projects/72145653914/locations/global/catalogs/default_catalog/branches/2/operations/import-products-6802980079078597073
Waiting for import operation to complete...
Import operation started. Operation name: projects/72145653914/locations/global/catalogs/default_catalog/branches/2/operations/import-products-6802980079078597073
Waiting for import operation to complete...
I

## Validation
Now that we have loaded the enriched product data into the Vertex AI Search for Commerce catalog, we can proceed with the validation process. We will use both automated and manual validation techniques. The validation tool will help us ensure that the enriched data meets the expected schema and values, and is ready for use in search and recommendation systems.

First we need to identify the products that we have enriched and loaded into the catalog. We will compare the results of catalog search performed againt Branch 0 (original product data) and Branch 2 (enriched product data). Let's identify search strings for relevant titles for us to perform the search with.

In [40]:
# Display 50 records from df_update where any extended attributes are not empty
non_empty_mask = (
    df_update['tags'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['patterns'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['sizes'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['brands'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['colorFamilies'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['colors'].apply(lambda x: isinstance(x, list) and len(x) > 0) |
    df_update['audience'].apply(lambda x: isinstance(x, dict) and (x.get('genders') or x.get('ageGroups')) and (len(x.get('genders', [])) > 0 or len(x.get('ageGroups', [])) > 0))
)
display(df_update[non_empty_mask].head(50))

,title,description,brands,tags,colorFamilies,colors,sizes,patterns,audience
0,Google Mural Tote,"Carry your essentials in style with the Google Mural Tote. Featuring a unique mural design, this tote is both fashionable and functional. Perfect for everyday use, it's a great way to show off your Google pride.",[Google],"[everyday, functional, google pride, mural design, stylish, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
1,Google Seaport Tote,"Show your Google spirit with the Google Seaport Tote. This stylish and durable tote bag is perfect for carrying your everyday essentials. Its spacious design and comfortable straps make it ideal for work, school, or travel.",[Google],"[comfortable, durable, everyday, google, school, spacious, tote bag, travel, work]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
2,Google Incognito Zippack V2,Keep your belongings secure and organized with the Google Incognito Zippack V2. This updated design offers enhanced security with its zippered compartments. Ideal for those who value privacy and functionality.,[Google],"[functional, organized, privacy, secure, zippered]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
3,Google Incognito Laptop Organizer V2,The Google Incognito Laptop Organizer V2 offers discreet protection and organization for your laptop and accessories. This updated version provides enhanced features and a sleek design. Perfect for professionals and students alike who need to keep their tech safe.,[Google],"[discreet, laptop, professional, sleek, student, tech]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
4,Google Mesh Bag Blue,"The Google Mesh Bag in Blue is a lightweight and breathable option for carrying your gear. Perfect for the gym, beach, or everyday errands. Its mesh design allows for ventilation and easy visibility of contents.",[Google],"[beach, breathable, everyday, gym, lightweight, mesh]",[Blue],[Blue],[],[],"{'genders': [], 'ageGroups': []}"
5,#IamRemarkable Tote,Celebrate the #IamRemarkable initiative with this empowering tote bag. Perfect for carrying your daily essentials while promoting self-promotion and confidence. Show your support and spread the message wherever you go.,[Unknown],"[confidence, empowering, iamremarkable, self-promotion, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
6,Google Campus Bike Carry Pouch,The Google Campus Bike Carry Pouch is designed for convenient storage while cycling. Securely carry your essentials on your bike with this durable and easy-to-install pouch. Perfect for campus commuters and urban cyclists.,[Google],"[bike, campus, convenient, cyclist, durable, easy-to-install]",[],[],[],[],"{'genders': [], 'ageGroups': []}"
7,Google Cork Pencil Pouch,"Store your writing essentials in sustainable style with the Google Cork Pencil Pouch. Made from eco-friendly cork, this pouch is both durable and unique. Keep your pens and pencils organized in a responsible way.",[Google],"[cork, durable, eco-friendly, pencil, sustainable, unique]",[Brown],[Cork],[],[],"{'genders': [], 'ageGroups': []}"
8,Google Campus Bike Tote Navy,"The Google Campus Bike Tote in Navy is designed for easy and stylish transportation on your bike. This tote securely attaches to your bike rack, making it perfect for carrying groceries, books, or other essentials. Enjoy a convenient and eco-friendly way to commute.",[Google],"[bike, commuting, easy transportation, eco-friendly, navy]",[Blue],[Navy],[],[],"{'genders': [], 'ageGroups': []}"
9,Google Cambridge Campus Tote,"Show your Cambridge campus pride with this durable and stylish tote. Perfect for carrying books, laptops, and other essentials around campus. Made with high-quality materials for long-lasting use.","[Google, cambridge]","[book, cambridge, durable, high-quality, laptop, stylish, tote bag]",[],[],[],[],"{'genders': [], 'ageGroups': []}"


While your results may very, for the products in this lab, you might find that the following search strings return the enriched products:
- "tote"
- "bag"
- "android phone"

To have visually appealing results, we will use the [Vertex AI Search for Commerce console](https://console.cloud.google.com/ai/retail/catalogs/default_catalog/evaluate/search) to perform the search. You can also use the python script to perform the search, but using the console allows us to easily visualize the results and compare them.

- naviate to the [Vertex AI Search for Commerce console](https://console.cloud.google.com/ai/retail/catalogs/default_catalog/evaluate/search)
- select `Branch 2` from the dropdown `Select catalog branch`
- enter the search string in the `Search query` field, use one of the search strings identified above
- click on the `Search` button

You should see the enriched products in the search results. To validate the advanced search attributes have been populated correctly, `select all` under `Facets` drop-down control. As you remember, we enabled site-wide controls to make all attributes are searchable, indexable and retrievable. Therefore, facets will be populated with the values from the enriched product data.

You should see something like this in the search results:
![Enhanced search results](https://raw.githubusercontent.com/volenin/vaisc-csb/refs/heads/main/notebooks/img/data_management_enchanced_results.png)

Now run the same same search against Branch 0 (original product data) and compare the results. You should see that the enriched products are not present in the search results for Branch 0, as they were not loaded into that branch.

![Basic search results](https://raw.githubusercontent.com/volenin/vaisc-csb/refs/heads/main/notebooks/img/data_management_basic_results.png)


# Conclusion
In this lab, we successfully demonstrated how to manage and enrich product data using Google Cloud services, particularly focusing on Vertex AI and BigQuery. We covered the following key steps:
-   **Data Analysis**: We analyzed the existing product data to identify gaps and opportunities for enrichment.
-   **Data Enrichment**: We utilized Generative AI models to extract structured information from product titles, generating attributes such as brand names, descriptions, tags, and color information.
-   **Data Population**: We created a new BigQuery table to store the enriched product data, ensuring that all original fields and newly generated attributes were included.
-   **Data Validation**: We validated the enriched product data using the Vertex AI Search for Commerce validation tool, ensuring that the data met our expectations and was ready for use in search and recommendation systems.

**Areas for Improvement**:
-   **Image Analysis**: In this lab, we focused on text data for simplicity. Future iterations could incorporate image analysis to extract additional attributes from product images, enhancing the catalog further.
-   **Attributes Cleansing**: while we consolidated tags and extracted colors, further cleansing and normalization of attributes could improve data quality. For example, ensuring consistent naming conventions for brands and colors, standardizing sizing, genders, and audience attributes.

**Operationalization**:
In this lab we demonstrated an approach to improve product data quality and searchability in iterative manner. The steps we followed can be operationalized in a production environment. One of the easiest ways to do this is to use the [BigQuery Pipelines](https://cloud.google.com/bigquery/docs/pipelines-introduction), as it supports direct execution of notebooks. Another approach is to transform the notebook into a Python script or PySpark jobs that can be scheduled to run periodically through Cloud Scheduler or BigQuery scheduled queries.

**Role of Generative AI**:
Generative AI played a crucial role in this lab by enabling us to extract structured information from unstructured text data. Not only that, the entire codebase in this notebook was generated using Gemini model, which demonstrates the power of Generative AI in automating and enhancing data management tasks. The code operationalization can also be done with help of Generative AI. 
